# [INFO-H515 - Big Data Scalable Analytics](https://uv.ulb.ac.be/course/view.php?id=85246?username=guest)

## TP 3 - Streaming forecasting (RLS and ML) with a network socket and Spark Streaming

#### *Gianluca Bontempi, Jacopo De Stefani and Theo Verhelst*

####  29/04/2020


## Streaming analytics with Spark streaming 

This notebook provides an example for performing machine learning analytics on streaming data coming from a network socket. 

The first part contains examples of simple __stateless__ processing scripts.

The second part introduces __stateful__ streamining processing in order to perform more complex operations, like cumulative sum, sequential estimation of mean and variance.

Predictive analytics is also implemented thanks to stateful processing.

Two main __streaming learning__ strategies are proposed: 
* an online linear learning where he coefficient estimation is achieved using the __recursive least square (RLS)__ algorithm
* a __mini-batch strategy__ where a learning model is fit to most recent data

A __racing__ strategy is implemented as well where two different models (e.g. two linear models with different forgetting factors or a linear and a nonlinear regressor) are fitted to the same data.

The data are produced in the notebook `TimeSeriesProducer`: two producers are considered, a linear one and a nonlinear one.

This notebook uses:
* [Spark streaming](https://spark.apache.org/docs/latest/streaming-programming-guide.html) for processing the streaming data
* A network socket for receiving data, as in this [example](https://spark.apache.org/docs/latest/streaming-programming-guide.html#a-quick-example)


## General imports

In [1]:
import time
import re, ast
import numpy as np
import os
import glob


from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn import linear_model
from pyspark.streaming import StreamingContext
from sklearn.linear_model import LinearRegression


### Start Spark session

Note: You should provide at least two cores to the Spark Context, as Spark Streaming requires one core for handling RDDs, the other cores being used for processing streaming data. 


In [2]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext


os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 '+\
        '--conf spark.driver.memory=2g  pyspark-shell '
    

spark = SparkSession \
    .builder \
    .master("local[4]") \
    .appName("StreamingAnalytics") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/25 08:34:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Function to connect Spark Streaming to a network socket

In [3]:
#This function creates a connection to a network socket
#You may change the batch interval
#The function returns the Spark context, Spark streaming context, and DStream object
def getDStream(spark,batch_interval=5):

    #Get Spark context
    sc=spark.sparkContext
    sc.setLogLevel("ERROR")

    #Create streaming context, with required batch interval
    ssc = StreamingContext(sc, batch_interval)

    #Checkpointing needed for stateful transforms
    ssc.checkpoint("checkpoint")
    
    # Create a DStream that represents streaming data from a network socket
    # See https://spark.apache.org/docs/latest/streaming-programming-guide.html#a-quick-example
    dstream = ssc.socketTextStream("localhost", 9999)
    
    return [sc,ssc,dstream]


## Raw data consumption

First, let's make sure that the data producer is running and ready to receive messages:

1. Look at the data format in the producer (`TimeSeriesProducer`) notebook. 
2. Start the Data production by running the corresponding cell in the producer notebook. 
3. Move to the consumer notebook (i.e. this notebook).

**N.B** You should see the connection to the client in the producer notebook by executing the cell below:

In [5]:
# Get the DStream object containing the streaming data sent by the producer notebook
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=1)
# Pretty print the content of dstream
dstream.pprint()

To start data consumption you have to launch the Dstream. This is equivalent to opening a data tap on the receiving side.

**Exercise:**

* Look at the time label: is it related to the batch interval you set? What about the data format?

In [6]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 07:43:39
-------------------------------------------

-------------------------------------------
Time: 2022-02-25 07:43:40
-------------------------------------------

-------------------------------------------
Time: 2022-02-25 07:43:41
-------------------------------------------

-------------------------------------------
Time: 2022-02-25 07:43:42
-------------------------------------------

-------------------------------------------
Time: 2022-02-25 07:43:43
-------------------------------------------

-------------------------------------------
Time: 2022-02-25 07:43:44
-------------------------------------------

-------------------------------------------
Time: 2022-02-25 07:43:45
-------------------------------------------

-------------------------------------------
Time: 2022-02-25 07:43:46
-------------------------------------------



-------------------------------------------
Time: 2022-02-25 07:43:47
-------------------------------------------
[ 0.        ,-0.69070552, 0.14116957,-0.35972144, 0.23525346,-0.84060208, 1.31307295, 0.21407109,-0.83490104, 0.32611369, 0.55512634,-0.87705984]
[ 1.        ,-1.19783834, 0.60197096, 1.23855365,-1.02675744,-0.13221217, 0.05266952, 0.31801924,-0.15956333, 0.48534895,-0.98568595,-1.82303797]
[ 2.        , 1.40743024, 0.89640743, 0.59349958, 1.23150731,-0.48190073,-0.83242817, 0.5046102 , 0.26038583, 0.20206037, 0.20707494, 0.43969144]
[ 3.        , 1.57545697, 0.48283181, 0.96588944,-2.71362256,-0.5599426 , 0.88906694,-0.98401467,-1.06144788, 0.30031018,-0.72943079, 1.00139819]
[ 4.        ,-0.15583584, 1.17130617,-0.63411887, 0.25766715,-0.15405764,-0.51831706,-1.12815054,-0.86595985,-0.17748257,-0.77464225,-1.34240026]
[ 5.        ,-0.12125809,-0.42194064, 1.07354404,-0.82047922,-1.45382133,-1.72097765,-0.41379372,-0.34063067,-0.95055884,-1.03530299, 0.29369309]
[ 6.      

-------------------------------------------
Time: 2022-02-25 07:43:49
-------------------------------------------
[ 1.55000000e+02,-4.59061738e-01,-5.50098172e-01, 8.61398305e-01, 5.02368579e-01,-2.50281098e-01,-3.81578962e-02, 1.04739546e+00,-8.39601814e-01, 2.50701670e+00, 8.43987088e-01, 7.25213926e-02]
[ 1.56000000e+02,-8.62725277e-01, 1.18156810e+00,-1.37440460e+00,-4.07078461e-01,-1.54540730e-01,-9.70861989e-01, 2.50102600e-01, 2.25100904e+00, 1.35841414e+00,-4.50437962e-01,-2.09201599e+00]
[ 1.57000000e+02,-1.56724265e-01, 7.25774574e-01, 5.30232704e-01,-1.46228933e+00,-1.46572030e+00,-6.78023792e-01, 9.07715006e-01, 6.25106171e-01,-1.63470787e-02,-8.97738360e-01,-9.74153167e-01]
[ 1.58000000e+02, 1.91908678e+00, 5.29228248e-01, 5.14828699e-02, 3.45258329e-01,-8.00604958e-01,-2.42117205e-01,-2.16880755e-01,-3.51193303e-01, 4.68240643e-01, 9.75029546e-01, 1.38731108e+00]
[ 1.59000000e+02,-7.17900730e-01, 7.29712741e-02, 1.54090357e-03, 7.56542648e-01, 1.70836134e+00, 3.37699097e-

-------------------------------------------
Time: 2022-02-25 07:43:50
-------------------------------------------
[ 2.39000000e+02,-8.12135840e-01,-7.15740708e-01,-9.17465731e-01, 8.46494535e-01, 1.30224372e+00,-2.49947508e-01, 7.63311527e-01, 1.20521835e+00, 5.49351826e-01, 2.41308028e-01,-1.31397708e-01]
[ 2.40000000e+02,-1.43133883e-01,-1.93518836e+00,-6.83515615e-01,-1.06633949e+00,-3.27378504e-01, 3.17532641e-01, 1.12781737e+00,-3.76379478e-02, 1.52630746e+00,-5.99925699e-01, 1.74421585e+00]
[ 2.41000000e+02, 6.15288610e-01, 7.20255974e-01, 5.03996502e-01,-2.06129939e+00,-8.33734198e-01,-4.27387961e-01, 1.86528125e+00, 1.01082907e+00,-7.15431893e-01,-2.88957862e-02,-1.83367367e-01]
[ 2.42000000e+02, 2.49122419e+00, 3.11586368e-01, 5.60439837e-01,-2.07242821e+00, 9.95408252e-01,-5.76257358e-01, 1.35536135e-02,-5.36182379e-01, 1.54517811e+00, 2.29494062e-01, 2.11205138e+00]
[ 2.43000000e+02, 1.96214659e-01,-4.14936593e-02,-9.02007353e-01,-1.66942212e+00,-1.94198255e+00, 2.96341779e-

-------------------------------------------
Time: 2022-02-25 07:43:51
-------------------------------------------
[ 3.21000000e+02,-4.52455708e-01, 2.00202432e+00,-7.35035030e-01,-1.10667947e+00, 1.45654417e+00,-5.55205061e-01,-2.08000914e+00, 1.15980210e-01, 7.51203173e-02,-7.51473220e-01,-2.49686651e+00]
[ 3.22000000e+02,-1.39025053e+00,-1.39850493e+00,-1.79333012e+00, 8.48271816e-01, 1.23553735e-02,-2.22741533e+00,-7.40212970e-01,-2.03754639e-01, 1.58670063e+00,-2.03413434e-01,-7.68760925e-02]
[ 3.23000000e+02,-3.23618899e+00,-2.95838357e+00,-2.28540218e-01,-1.76111019e-01, 3.36956139e-01, 1.92198148e+00, 6.21548382e-01,-1.08789568e+00, 1.85856129e-01,-9.05834892e-01,-3.29776938e-01]
[ 3.24000000e+02, 4.27076414e-01,-3.54433432e-01, 8.62213614e-01, 1.88400039e+00,-2.74187991e-01, 4.95944465e-01,-4.36816665e-01, 5.61182748e-03,-1.09403492e+00,-1.29776062e+00, 6.97525034e-01]
[ 3.25000000e+02,-3.94155290e+00,-2.03758232e+00,-1.62168261e-02,-5.98792058e-01, 1.27468757e+00,-2.33216059e-

...

-------------------------------------------
Time: 2022-02-25 07:44:21
-------------------------------------------
[16.        , 0.49769238, 0.54788808,-2.06151586,-0.97663307,-1.41541278, 1.72345906, 0.1651739 ,-2.05023964,-0.36905598,-0.22961612,-0.06856231]



-------------------------------------------
Time: 2022-02-25 07:44:22
-------------------------------------------
[17.        , 2.55343342, 1.3980414 , 0.23989253, 1.1331776 ,-0.44162287,-0.63741667,-1.17204765,-1.20268201, 1.90072485,-1.26976197, 1.06076802]



-------------------------------------------
Time: 2022-02-25 07:44:23
-------------------------------------------
[ 1.80000000e+01, 1.02278638e+00,-5.48766625e-01,-2.65867526e-01, 8.92682412e-01, 6.70673205e-01, 4.94819011e-03, 7.92984244e-01, 7.27769259e-01, 1.27197515e+00, 1.24893763e+00, 1.57033012e+00]



-------------------------------------------
Time: 2022-02-25 07:44:24
-------------------------------------------
[19.        , 1.11033051, 1.19008374,-0.46983116, 0.5042198 ,-0.41693386, 0.94836584,-0.38823477,-0.63969231, 0.32197598, 0.44609022,-0.1766717 ]



-------------------------------------------
Time: 2022-02-25 07:44:25
-------------------------------------------
[20.        ,-1.76234465,-0.36918268,-0.66981411, 1.16221588,-0.80063791,-0.38988467,-0.96630729, 1.71599918, 0.52078762,-0.14982941,-1.49279394]



To stop data reception you have to stop the Dstream (close the tap).

In [7]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 07:44:25 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:3

**Tip:** Clear all outputs before starting the Streaming again with ssc.start()! 

Note: You can get an Exception ('ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver'), due to the closing of the stream by the driver. This is more a warning message than an exception.

**Exercise:**

* Change the batch interval value (in the consumer) and observe the impact on the buffer. What can you deduce?

# Stateless processing

The first examples in this notebook will introduce the concept of **stateless** processing.

In this case, the Spark transformations are applied directly to the Discretized Stream (Dstream) produced by the network socket.

A Discretized Stream (cf. [Spark Streaming Programming guide](https://spark.apache.org/docs/latest/streaming-programming-guide.html)) is simply a continuous collection of Spark RDD.

Applying a transformation to a Dstream basically boils down to applying the same transformation to each underlying RDD.

## Converting the data to a suitable format

Before being able to actually perform a transformation, we need to de-serialize the data obtained from the network socket.
Since we want to do numerical processing, the raw data format is not suitable. 
Let's see an example of data conversion to a suitable numeric format.

In [8]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=2)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))
dataS.pprint()

In [9]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 08:36:28
-------------------------------------------
[ 2.65220000e+04 -5.91845193e-01  7.92990413e-01 -7.88462689e-01
  7.36209072e-01  6.33357068e-02  1.89767763e+00 -1.06092282e+00
  3.68019525e-01  1.31677570e+00 -1.12729563e-01 -1.42180687e+00]
[ 2.65230000e+04  8.84059329e-01  6.74094167e-01 -2.01955093e-02
 -1.59200743e+00 -7.21803477e-01  2.99014069e-01 -4.93739300e-01
  3.87461730e-01  7.05171243e-01  1.20613799e+00  2.08858309e-01]
[ 2.65240000e+04 -3.07084774e+00 -5.67687588e-01 -4.23387799e-01
  8.39444191e-01 -1.45112189e+00  3.72287387e-01 -4.36969664e-01
  1.11748836e+00 -6.10214514e-01  7.71874025e-01 -2.57792308e+00]
[ 2.65250000e+04 -6.94281695e-01  2.95261859e-01  6.62133449e-01
 -1.87552695e-01 -2.36723739e-01  1.26373008e+00 -1.27445039e-01
 -1.12434893e+00  4.19707100e-01  6.62270907e-01 -1.01181350e+00]
[ 2.65260000e+04  9.27463917e-01  8.15173411e-01 -5.94987660e-01
  1.20919468e+00 -2.45900131e+00  4.1

**Exercise:**

* Compare the suitable data format with the previous format. What is the main difference?


In [10]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 08:36:29 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:3

## Applying a map transformation to a Dstream

The examples below apply a map transformation to each element of the batch without storing any state information.

In [12]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=5)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))
dataS.pprint()
maxS=dataS.map(lambda x: np.max(x[1:]))
maxS.pprint()

In [13]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 07:48:30
-------------------------------------------
[30.          1.53846461 -0.19715228  0.56043625 -1.54592393  0.24277956
  0.12708401 -0.41835225 -0.33377962 -0.77420308  0.65217619  1.63750737]
[ 3.10000000e+01 -6.70739689e-01 -9.31358355e-01  1.50281610e+00
 -3.68239857e-01 -9.89989862e-01  1.32981456e+00 -6.23494642e-01
 -2.38787797e-02  2.11776974e+00 -1.15502232e+00  1.83637385e-01]

-------------------------------------------
Time: 2022-02-25 07:48:30
-------------------------------------------
1.63750737
2.11776974



-------------------------------------------
Time: 2022-02-25 07:48:35
-------------------------------------------
[32.         -1.46813941 -1.52113158  0.594973    0.39877172 -1.05230309
  1.92029748 -0.48338847  0.80639621 -1.87209897 -1.33323131  0.05003964]
[ 3.30000000e+01  4.46642364e-01  2.69253389e-02 -1.86928673e+00
 -1.00549579e-01  2.06320115e+00  3.70698896e-01  1.93980990e+00
 -6.98949589e-01 -5.35240151e-01 -8.04816137e-01  3.47671273e-01]
[34.         -0.56878965 -0.13357493  0.4652181  -0.48595613  1.58305067
 -2.05536534  1.47795204 -0.67225941  1.17243371 -1.36225377 -0.45909414]
[35.          0.49369315  0.09782546 -0.23760337 -0.54436411  1.59405082
 -0.93444112 -0.41550651  0.94927623 -0.04590219  0.36679324  0.37408697]
[ 3.60000000e+01  3.81107026e-01 -3.53712015e-02  3.05542569e-01
  3.46670864e-01  7.78995388e-01  1.61481252e+00  9.27839335e-02
 -3.58483684e-01  1.61762014e+00  8.55103559e-01  4.12676347e-01]

-------------------------------------------
Time: 20

**Exercise:**

* Compare the content of the Dstream before and after the transformation. What is the applied transformation?


In [14]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 07:48:37 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:3

**Exercise:**

* Adapt the code of the previous exercise in order to compute the mean of the $x_i$ values at each time step.


In [15]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=2)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))
dataS.pprint()
maxS=dataS.map(lambda x: np.mean(x[1:]))
maxS.pprint()

In [16]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 07:49:00
-------------------------------------------
[ 4.20000000e+01  8.22155850e-01  7.27186697e-01 -1.59441236e+00
 -3.67009835e-01  2.80549400e-01 -1.84679345e+00 -1.04109271e+00
  7.30869951e-01  1.30328627e+00 -7.91409832e-01  3.60849696e-02]

-------------------------------------------
Time: 2022-02-25 07:49:00
-------------------------------------------
-0.15823500449090908



-------------------------------------------
Time: 2022-02-25 07:49:02
-------------------------------------------
[ 4.30000000e+01 -1.67367910e-02 -3.52483663e-01  4.13161393e-01
  7.19414353e-01  2.10858787e+00 -1.05508827e+00 -1.68401782e+00
 -2.56610040e-01  7.67110197e-01 -5.31519208e-01  3.23362828e-01]
[44.          2.2882212   2.16638519 -1.70243109  0.4333817   0.19358709
  0.55275183 -1.58165928 -0.33203224  0.88335618 -1.52338005  0.12039313]

-------------------------------------------
Time: 2022-02-25 07:49:02
-------------------------------------------
0.039561895363636385
0.1362339690909091



In [17]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 07:49:03 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:3

## Custom presentation of the received information 

Here you can find an example of using a `map` transformation to add some descriptive text to identify the received values.

In [18]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=2)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))
dataS.map(lambda x: "Stream counter : "+ str(x[0])).pprint()


In [19]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 07:49:30
-------------------------------------------



-------------------------------------------
Time: 2022-02-25 07:49:32
-------------------------------------------
Stream counter : 49.0
Stream counter : 50.0



-------------------------------------------
Time: 2022-02-25 07:49:34
-------------------------------------------
Stream counter : 51.0
Stream counter : 52.0



-------------------------------------------
Time: 2022-02-25 07:49:36
-------------------------------------------
Stream counter : 53.0
Stream counter : 54.0



-------------------------------------------
Time: 2022-02-25 07:49:38
-------------------------------------------
Stream counter : 55.0
Stream counter : 56.0



-------------------------------------------
Time: 2022-02-25 07:49:40
-------------------------------------------
Stream counter : 57.0
Stream counter : 58.0



-------------------------------------------
Time: 2022-02-25 07:49:42
-------------------------------------------
Stream counter : 59.0
Stream counter : 60.0



-------------------------------------------
Time: 2022-02-25 07:49:44
-------------------------------------------
Stream counter : 61.0
Stream counter : 62.0



As the first element of the transmitted data is the counter value, we can see which line is currently sent by the producer. If you do not stop the producer this value will continue growing. Given the stationary nature of our data set, this has not much of an impact for the following analysis....

In [20]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 07:49:45 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


-------------------------------------------
Time: 2022-02-25 07:49:46
-------------------------------------------
Stream counter : 63.0
Stream counter : 64.0



Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurren

## Output to the disk

To save the content of the Dstream to the disk you can use the Spark function `saveAsTextFiles(prefix, [suffix])`.

The file name at each batch interval is generated based on the prefix and suffix parameters, as well as the current time in milliseconds: `prefix-<TIME_IN_MS>[.suffix]`.

**Warning:** This command creates a new file for each batch: if the `batch_interval` value is low this can fill your directory rapidly.

In [21]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=10)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))
dataS.map(lambda x: "stream counter="+ str(x[0])).pprint()
prefix='test'
dataS.map(lambda x: "stream counter="+ str(x[0])).saveAsTextFiles(prefix)


Let the streaming run for two epochs only and then look at your current directory.

In [22]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 07:50:10
-------------------------------------------
stream counter=67.0
stream counter=68.0
stream counter=69.0
stream counter=70.0
stream counter=71.0
stream counter=72.0



-------------------------------------------
Time: 2022-02-25 07:50:20
-------------------------------------------
stream counter=73.0
stream counter=74.0
stream counter=75.0
stream counter=76.0
stream counter=77.0
stream counter=78.0
stream counter=79.0
stream counter=80.0
stream counter=81.0
stream counter=82.0



In [23]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 07:50:21 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:3

Here you have an example on how to visualize the directories containing the content of the processed batch 

In [24]:
search_dir = "./"
files = list(filter(os.path.isdir, glob.glob(search_dir + prefix +"*")))
files.sort(key=lambda x: os.path.getmtime(x))
files

['./test-1645697970000',
 './test-1645698100000',
 './test-1645698110000',
 './test-1645698120000',
 './test-1645698270000',
 './test-1645698280000',
 './test-1645698290000',
 './test-1645698300000',
 './test-1645698310000',
 './test-1645698320000',
 './test-1645698330000',
 './test-1645698340000',
 './test-1645698350000',
 './test-1645698360000',
 './test-1645698370000',
 './test-1645775410000',
 './test-1645775420000']

Note that each directory is splitted in several parts.

In [25]:
fyhat=files[0]+"/part-00000"
F=open(fyhat, "r")
F.read().replace("\n",", ")

'stream counter=107.0, '

In [26]:
fyhat=files[0]+"/part-00001"
F=open(fyhat, "r")
F.read().replace("\n",", ")

'stream counter=108.0, '

# Stateful processing

As opposed to **stateless** processing, **stateful** process implies the existence of a state.

According to the [Spark Streaming Programming guide](https://spark.apache.org/docs/latest/streaming-programming-guide.html), the key concept for **stateful** processing is the `updateStateByKey`:

>The `updateStateByKey` operation allows you to maintain arbitrary state while continuously updating it with new information. To use this, you will have to do two steps.
>
>    1. *Define the state* - The state can be an arbitrary data type.
>    2. *Define the state update function* - Specify with a function how to update the state using the previous state and the new values from an input stream.

The following examples will clarify how to create and initialize the state and how to perform updates of the state through the combination of `updateStateByKey` and user-defined functions `updateFunctions`.
First example of stateful streaming processing obtained with an `updateFunction` called by the command __updateStateByKey__.

## Single state value - No memory of the state

In this first example, you will learn how to define the state as well as to code a simple `updateFunction` which takes into account only the first value in the streaming buffer and which does not perform a state update.

Let's start by defining the state:

In [27]:
# Initialize an empty vector with n elements
n=11
v=np.zeros((1,n)) 

state1=(v,0) # State is a 2 element tuple: (vector of size n,integer)

# Transform the state into an RDD
initialStateRDD = sc.parallelize([('state', state1)]) 

Let's continue by defining the user defined `updateFunction`.

Note that all the update functions in the following cells share the same structure:

```python 
def updateFunction(new_values, state): 
    L=len(new_values) # Size of the message buffer
    if (L>0): # If there are new values in the buffer,
        (...) = state # Unpacking the state
        # Perform state update
        # ...
        # ...
        return new_state  # Then return a new value for the state
    else: # If there are no new values
        return state # then return the unmodified state
```

In [28]:
def updateFunction(new_values, state):
    L=len(new_values) # size buffer
    if (L>0):
        (d, N) = state # Unpacking the state
        value=new_values[0] ## take only the first value
        d=np.abs(value[1:]).reshape(1,-1) ## d does NOT depend on the 'state' input
        return (d,L)  ## return the value and size buffer
        
    else:
        return state

Finally, let's conclude by applying the `updateFunction` on the Spark Streaming Dstream by means of the `updateStateByKey` operation.

In [29]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=2)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))
dataS.pprint()

dataS=dataS.flatMap(lambda x: [('state',1.0*np.array(x))])

updatedS=dataS.updateStateByKey(updateFunction,initialRDD=initialStateRDD)
updatedS.pprint()


In [30]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 08:00:16
-------------------------------------------
[87.         -1.10112374 -0.5708041  -0.37645256  1.32963664  0.198458
 -1.01439443 -0.22719201 -0.75834343 -1.1896548   0.36520875 -0.57796833]

-------------------------------------------
Time: 2022-02-25 08:00:16
-------------------------------------------
('state', (array([[1.10112374, 0.5708041 , 0.37645256, 1.32963664, 0.198458  ,
        1.01439443, 0.22719201, 0.75834343, 1.1896548 , 0.36520875,
        0.57796833]]), 1))



-------------------------------------------
Time: 2022-02-25 08:00:18
-------------------------------------------
[ 8.80000000e+01 -6.28653048e-01  4.69537479e-02 -7.06156200e-01
  9.69392145e-01 -1.71373793e-01  6.49347619e-01 -4.60285227e-01
  4.72191903e-01  3.88307464e-01  5.41773466e-01 -7.60867710e-01]
[ 8.90000000e+01  5.20503195e-02 -2.98077153e-01 -5.19708270e-01
  2.57559206e+00 -7.62739342e-01  1.17428514e+00  1.12568717e-01
 -2.47998691e-01  1.03359879e+00 -1.14372585e-01  3.08898325e-01]

-------------------------------------------
Time: 2022-02-25 08:00:18
-------------------------------------------
('state', (array([[0.62865305, 0.04695375, 0.7061562 , 0.96939215, 0.17137379,
        0.64934762, 0.46028523, 0.4721919 , 0.38830746, 0.54177347,
        0.76086771]]), 2))



**Exercise:**

* Compare the raw data and the content of the state. What can you observe?


In [31]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 08:00:20 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


-------------------------------------------
Time: 2022-02-25 08:00:20
-------------------------------------------
[ 9.00000000e+01 -6.60698827e-02  6.41585918e-01 -6.68539199e-01
  3.03182503e-01 -3.24030650e-02 -3.29184468e-01 -1.33369234e+00
  5.41062894e-01 -6.02306419e-01  4.19486766e-01 -7.30225011e-01]
[ 9.10000000e+01  3.58775093e-01  5.23709092e-02 -1.31983367e+00
  1.15352401e+00 -5.46233545e-01  1.54334117e-01 -3.87891161e-02
  4.00902544e-01  6.29853316e-01  1.62732926e+00  2.84259639e-01]

-------------------------------------------
Time: 2022-02-25 08:00:20
-------------------------------------------
('state', (array([[0.06606988, 0.64158592, 0.6685392 , 0.3031825 , 0.03240307,
        0.32918447, 1.33369234, 0.54106289, 0.60230642, 0.41948677,
        0.73022501]]), 2))



Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurren

## Single state value - With memory of the state

In this case the output of the function does not depend only on the current values in the buffer but also on the previous state. 

Let's start with defining the state and the `updateFunction`:

In [32]:
# Initialize an empty vector with n elements
n=11
v=np.zeros((1,n)) 

state1=(v,0) # State is a 2 element tuple: (vector of size n,integer)

# Transform the state into an RDD
initialStateRDD = sc.parallelize([('state', state1)]) 

In [33]:
def updateFunction(new_values, state): 
    ## Sum of absolute values of all vectors in the buffer
    L=len(new_values)
    if (L>0):
        for l in np.arange(L):
            value=new_values[l]
            d=state[0]+np.abs(value[1:]).reshape(1,-1) ### d DOES depend  on state !!
        return (d,L)  
        
    else:
        return state

**Exercise:**

* Compare this `updateFunction` with the one in the section before. What can you observe?
* What operation is being performed inside this `updateFunction`?
* Run the code below to check your answers.

**Solution:**
* Here 'd' depends on the state but it was NOT the case before.
* This is due to the fact that I want to compute a cumulative sum which does not depend only on the current values but also on the past ones (stored in the state variable).

In [34]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=2)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))
dataS.pprint()

dataS=dataS.flatMap(lambda x: [('state',1.0*np.array(x))])

updatedS=dataS.updateStateByKey(updateFunction,initialRDD=initialStateRDD)
updatedS.pprint()

In [35]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 08:01:50
-------------------------------------------



-------------------------------------------
Time: 2022-02-25 08:01:50
-------------------------------------------
('state', (array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), 0))



-------------------------------------------
Time: 2022-02-25 08:01:52
-------------------------------------------
[ 9.40000000e+01  3.99565820e-01  7.67266786e-02  6.82652931e-02
  1.86674510e+00 -1.18663372e+00 -1.24793436e+00  1.06961332e+00
 -4.55793507e-01  2.57876977e-02  9.28240406e-01  2.73730385e-01]
[95.          3.48151967  2.16159317 -0.46641652 -1.02316039  0.2401947
 -0.49775607 -0.60255872 -0.4955465   1.04487247  0.36857549  1.279245  ]

-------------------------------------------
Time: 2022-02-25 08:01:52
-------------------------------------------
('state', (array([[3.48151967, 2.16159317, 0.46641652, 1.02316039, 0.2401947 ,
        0.49775607, 0.60255872, 0.4955465 , 1.04487247, 0.36857549,
        1.279245  ]]), 2))



-------------------------------------------
Time: 2022-02-25 08:01:54
-------------------------------------------
[96.          2.15209098  0.86295012  0.29736268  1.41075741  0.23592833
 -0.43420576 -1.4577013  -0.13055077 -0.15944824 -0.6334045   1.28051158]
[ 9.70000000e+01  5.60642647e-01  7.24871062e-01 -9.46757435e-02
 -1.05338870e+00 -1.16838592e+00 -4.05232057e-01  1.38851980e+00
 -2.03439254e-01 -5.67039346e-01 -2.19391825e+00 -2.32858567e-01]

-------------------------------------------
Time: 2022-02-25 08:01:54
-------------------------------------------
('state', (array([[4.04216232, 2.88646423, 0.56109226, 2.07654909, 1.40858062,
        0.90298813, 1.99107852, 0.69898575, 1.61191182, 2.56249374,
        1.51210357]]), 2))



In [36]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 08:01:54 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:3

## Multiple values - with memory of the state

Let's see here an example of usage of multiple values in the buffer (as opposed to the previous examples, only using the last value in the buffer).

Note that the structure of the `updateFunction` remains the same as before, with the addition of a loop for iterating through the different elements of the buffer `new_values`.

```python 
def updateFunction(new_values, state): 
    L=len(new_values) # Size of the message buffer
    if (L>0): # If there are new values in the buffer,
        (...) = state # Unpacking the state
        for l in np.arange(L):
            value=new_values[l]
            # Perform state update
            # ...
            # ...
        return new_state  # Then return the updated state
    else: # If there are no new values
        return state # then return the unmodified state
```

In this case, all the different values of the state are stacked in a matrix.
Beware that, with this implementation, the size of this matrix keeps growing indefinitely.

In [69]:
# Initialize an empty vector with n elements
n=11
v=np.zeros((1,n)) 

state1=(v,0) # State is a 2 element tuple: (vector of size n,integer)

# Transform the state into an RDD
initialStateRDD = sc.parallelize([('state', state1)]) 

In [70]:
def updateFunction(new_values, state):
    L=len(new_values)  ## size of the buffer
    if (L>0):
        D=state[0]
        for l in np.arange(L): ## loop over all the values of the buffer
            value=new_values[l]
            D=np.vstack((D,value[1:].reshape(1,-1)))
        return (D,D.shape,L)  
        
    else:
        return state

In [71]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=2)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))
dataS.pprint()

dataS=dataS.flatMap(lambda x: [('state',1.0*np.array(x))])

updatedS=dataS.updateStateByKey(updateFunction,initialRDD=initialStateRDD)
updatedS.pprint()

In [72]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 08:16:18
-------------------------------------------



-------------------------------------------
Time: 2022-02-25 08:16:18
-------------------------------------------
('state', (array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), 0))



-------------------------------------------
Time: 2022-02-25 08:16:20
-------------------------------------------
[19.          1.11033051  1.19008374 -0.46983116  0.5042198  -0.41693386
  0.94836584 -0.38823477 -0.63969231  0.32197598  0.44609022 -0.1766717 ]
[20.         -1.76234465 -0.36918268 -0.66981411  1.16221588 -0.80063791
 -0.38988467 -0.96630729  1.71599918  0.52078762 -0.14982941 -1.49279394]

-------------------------------------------
Time: 2022-02-25 08:16:20
-------------------------------------------
('state', (array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.11033051,  1.19008374, -0.46983116,  0.5042198 , -0.41693386,
         0.94836584, -0.38823477, -0.63969231,  0.32197598,  0.44609022,
        -0.1766717 ],
       [-1.76234465, -0.36918268, -0.66981411,  1.16221588, -0.80063791,
        -0.38988467, -0.96630729,  1.71599918,  0.5207876

-------------------------------------------
Time: 2022-02-25 08:16:22
-------------------------------------------
[21.         -2.54562735 -2.47728489 -2.96352031 -0.30727978  1.89335993
  1.93205886  0.24049817 -1.66017067 -0.09230093  0.67272552 -0.16283446]
[ 2.20000000e+01  3.91860844e-01  9.22793690e-02 -1.64863753e-02
  1.33473070e+00 -1.04149226e+00 -4.70219017e-01 -1.73829331e-01
 -1.19239569e+00 -1.47387269e-01 -3.14013413e-01  2.07227440e-01]

-------------------------------------------
Time: 2022-02-25 08:16:22
-------------------------------------------
('state', (array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.11033051,  1.19008374, -0.46983116,  0.5042198 , -0.41693386,
         0.94836584, -0.38823477, -0.63969231,  0.32197598,  0.44609022,
        -0.1766717 ],
       [-1.76234465, -0.36918268, -0.66981411,  1.16221588, -0.80063791,
        

In [73]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 08:16:22 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:3

## Sequential estimation of sampled means and variances by state update


A more complex example of stateful processing is given by the sequential estimation of the mean of a random variable.

Average batch formulation: $\mu_{(N)}=\frac{1}{N} \sum_{i=1}^N z_i $

Average sequential formulation:  $\mu_{(N)}=\mu_{(N-1)} +\frac{1}{N} (z_N -\mu_{(N-1)} ) $


In [103]:
muhat=0
N=0
state1=(muhat,N)
## state with two components: estimate of mu and number of samples

initialStateRDD = sc.parallelize([('state', state1)])

In [104]:
def updateFunction(new_values, state): 
    ## Update the sequential estimate of sample mean of x[0] 
    ## (i.e. second element of value vector, the first being the counter)
    L=len(new_values) ## size of the buffer
    if (L>0):
        (muhat,N) = state # Unpacking the state
        for l in np.arange(L):
            N=N+1
            value=new_values[l]
            muhat=muhat+1.0/N*(value[2]-muhat)
        return (muhat,N)    
    else:
        return state

In [105]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=2)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))
dataS.map(lambda x: "stream counter="+ str(x[0])).pprint()

dataS=dataS.flatMap(lambda x: [('state',1.0*np.array(x))])

updatedS=dataS.updateStateByKey(updateFunction,initialRDD=initialStateRDD)
updatedS.map(lambda x : 'After '+str(x[1][1])+' samples: muhat='+str(x[1][0])).pprint()

In [106]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 08:27:56
-------------------------------------------
stream counter=2815.0
stream counter=2816.0
stream counter=2817.0
stream counter=2818.0
stream counter=2819.0
stream counter=2820.0
stream counter=2821.0
stream counter=2822.0
stream counter=2823.0
stream counter=2824.0
...

-------------------------------------------
Time: 2022-02-25 08:27:56
-------------------------------------------
After 62 samples: muhat=0.1682240555177419



-------------------------------------------
Time: 2022-02-25 08:27:58
-------------------------------------------
stream counter=2877.0
stream counter=2878.0
stream counter=2879.0
stream counter=2880.0
stream counter=2881.0
stream counter=2882.0
stream counter=2883.0
stream counter=2884.0
stream counter=2885.0
stream counter=2886.0
...

-------------------------------------------
Time: 2022-02-25 08:27:58
-------------------------------------------
After 230 samples: muhat=0.08380666053197383



-------------------------------------------
Time: 2022-02-25 08:28:00
-------------------------------------------
stream counter=3045.0
stream counter=3046.0
stream counter=3047.0
stream counter=3048.0
stream counter=3049.0
stream counter=3050.0
stream counter=3051.0
stream counter=3052.0
stream counter=3053.0
stream counter=3054.0
...

-------------------------------------------
Time: 2022-02-25 08:28:00
-------------------------------------------
After 398 samples: muhat=0.013829865181291418



-------------------------------------------
Time: 2022-02-25 08:28:02
-------------------------------------------
stream counter=3213.0
stream counter=3214.0
stream counter=3215.0
stream counter=3216.0
stream counter=3217.0
stream counter=3218.0
stream counter=3219.0
stream counter=3220.0
stream counter=3221.0
stream counter=3222.0
...

-------------------------------------------
Time: 2022-02-25 08:28:02
-------------------------------------------
After 566 samples: muhat=0.0368379518764911



-------------------------------------------
Time: 2022-02-25 08:28:04
-------------------------------------------
stream counter=3381.0
stream counter=3382.0
stream counter=3383.0
stream counter=3384.0
stream counter=3385.0
stream counter=3386.0
stream counter=3387.0
stream counter=3388.0
stream counter=3389.0
stream counter=3390.0
...

-------------------------------------------
Time: 2022-02-25 08:28:04
-------------------------------------------
After 735 samples: muhat=0.016538634209703352



...

-------------------------------------------
Time: 2022-02-25 08:28:14
-------------------------------------------
stream counter=4237.0
stream counter=4238.0
stream counter=4239.0
stream counter=4240.0
stream counter=4241.0
stream counter=4242.0
stream counter=4243.0
stream counter=4244.0
stream counter=4245.0
stream counter=4246.0
...

-------------------------------------------
Time: 2022-02-25 08:28:14
-------------------------------------------
After 1591 samples: muhat=0.004929715698235037



-------------------------------------------
Time: 2022-02-25 08:28:16
-------------------------------------------
stream counter=4406.0
stream counter=4407.0
stream counter=4408.0
stream counter=4409.0
stream counter=4410.0
stream counter=4411.0
stream counter=4412.0
stream counter=4413.0
stream counter=4414.0
stream counter=4415.0
...

-------------------------------------------
Time: 2022-02-25 08:28:16
-------------------------------------------
After 1761 samples: muhat=-0.002873178763349266



-------------------------------------------
Time: 2022-02-25 08:28:18
-------------------------------------------
stream counter=4576.0
stream counter=4577.0
stream counter=4578.0
stream counter=4579.0
stream counter=4580.0
stream counter=4581.0
stream counter=4582.0
stream counter=4583.0
stream counter=4584.0
stream counter=4585.0
...

-------------------------------------------
Time: 2022-02-25 08:28:18
-------------------------------------------
After 1927 samples: muhat=-0.0014564869468438287



-------------------------------------------
Time: 2022-02-25 08:28:20
-------------------------------------------
stream counter=4742.0
stream counter=4743.0
stream counter=4744.0
stream counter=4745.0
stream counter=4746.0
stream counter=4747.0
stream counter=4748.0
stream counter=4749.0
stream counter=4750.0
stream counter=4751.0
...

-------------------------------------------
Time: 2022-02-25 08:28:20
-------------------------------------------
After 2093 samples: muhat=-0.0009638612966689249



-------------------------------------------
Time: 2022-02-25 08:28:22
-------------------------------------------
stream counter=4908.0
stream counter=4909.0
stream counter=4910.0
stream counter=4911.0
stream counter=4912.0
stream counter=4913.0
stream counter=4914.0
stream counter=4915.0
stream counter=4916.0
stream counter=4917.0
...

-------------------------------------------
Time: 2022-02-25 08:28:22
-------------------------------------------
After 2263 samples: muhat=0.0014210538254922408



**N.B.**: With the dataLinearModel topic, mean estimation must converge to 0.

In [107]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 08:28:22 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:3

**Exercise**

Modify the state and the update function to perform the sequential computation of the variance. 

The state should contain two additional variables:  $\sigma^2_{(N)}$ and $S_{(N)}$.

The formulas definining the state update are the following ones:

- Sample variance batch formulation: $\sigma^2_{(N)}=\frac{1}{N} \sum_{i=1}^N (z_i -\mu_{(N)})^2= 
\left(\frac{1}{N} \sum_{i=1}^N z_i^2 \right) -  \left(\frac{1}{N} \sum_{i=1}^N z_i \right) ^2 $

- Sample variance sequential formulation:  $\sigma^2_{(N)} ={S_{(N)}/N}$ where $$S_{(N)}= S_{(N-1)}+N(N-1) (\mu_{(N)}- \mu_{(N-1)})^2 $$ 

**Solution**

In [108]:
muhat=0
sigma2hat=0
Shat=0
N=0
state1=(muhat,sigma2hat,Shat,N)
## state with four components: estimate of mu, estimate of sigma2hat, sumS and number of samples

initialStateRDD = sc.parallelize([('state', state1)])


In [109]:
def updateFunction(new_values, state): 
    ## Update the sequential estimate of sample mean and sample variance of x[0]
    ## (i.e. second element of value vector, the first being the counter)
    L=len(new_values) ## size of the buffer
    if (L>0):
        (muhat,sigma2hat,Shat,N) = state # Unpacking the state
        for l in np.arange(L):
            N=N+1
            value=new_values[l]
            muhatold=muhat
            muhat=muhat+1.0/N*(value[2]-muhat)
            Shat=Shat+N*(N-1)*pow(muhat-muhatold,2.0)
            sigma2hat=Shat/N
        return (muhat,sigma2hat,Shat,N)  
        
    else:
        return state

In [113]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=2)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))
dataS.map(lambda x: "stream counter="+ str(x[0])).pprint()

dataS=dataS.flatMap(lambda x: [('state',1.0*np.array(x))])

updatedS=dataS.updateStateByKey(updateFunction,initialRDD=initialStateRDD)
updatedS.map(lambda x : 'After '+str(x[1][3])+' samples: muhat='+str(x[1][0])+' varhat='+ str(x[1][1])).pprint()

In [114]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 08:28:36
-------------------------------------------

-------------------------------------------
Time: 2022-02-25 08:28:36
-------------------------------------------
After 0 samples: muhat=0 varhat=0



-------------------------------------------
Time: 2022-02-25 08:28:38
-------------------------------------------
stream counter=5272.0
stream counter=5273.0
stream counter=5274.0
stream counter=5275.0
stream counter=5276.0
stream counter=5277.0
stream counter=5278.0
stream counter=5279.0
stream counter=5280.0
stream counter=5281.0
...

-------------------------------------------
Time: 2022-02-25 08:28:38
-------------------------------------------
After 164 samples: muhat=0.15718226081329267 varhat=1.0893719491536635



-------------------------------------------
Time: 2022-02-25 08:28:40
-------------------------------------------
stream counter=5436.0
stream counter=5437.0
stream counter=5438.0
stream counter=5439.0
stream counter=5440.0
stream counter=5441.0
stream counter=5442.0
stream counter=5443.0
stream counter=5444.0
stream counter=5445.0
...

-------------------------------------------
Time: 2022-02-25 08:28:40
-------------------------------------------
After 333 samples: muhat=0.05561355891804798 varhat=1.0093725056056067



-------------------------------------------
Time: 2022-02-25 08:28:42
-------------------------------------------
stream counter=5605.0
stream counter=5606.0
stream counter=5607.0
stream counter=5608.0
stream counter=5609.0
stream counter=5610.0
stream counter=5611.0
stream counter=5612.0
stream counter=5613.0
stream counter=5614.0
...

-------------------------------------------
Time: 2022-02-25 08:28:42
-------------------------------------------
After 503 samples: muhat=0.014631138749383597 varhat=0.9742273474183952



-------------------------------------------
Time: 2022-02-25 08:28:44
-------------------------------------------
stream counter=5775.0
stream counter=5776.0
stream counter=5777.0
stream counter=5778.0
stream counter=5779.0
stream counter=5780.0
stream counter=5781.0
stream counter=5782.0
stream counter=5783.0
stream counter=5784.0
...

-------------------------------------------
Time: 2022-02-25 08:28:44
-------------------------------------------
After 680 samples: muhat=0.029577108831235228 varhat=0.9446871463693645



...

-------------------------------------------
Time: 2022-02-25 08:28:50
-------------------------------------------
stream counter=6287.0
stream counter=6288.0
stream counter=6289.0
stream counter=6290.0
stream counter=6291.0
stream counter=6292.0
stream counter=6293.0
stream counter=6294.0
stream counter=6295.0
stream counter=6296.0
...

-------------------------------------------
Time: 2022-02-25 08:28:50
-------------------------------------------
After 1181 samples: muhat=0.044912620545845834 varhat=0.9726982352389033



-------------------------------------------
Time: 2022-02-25 08:28:52
-------------------------------------------
stream counter=6453.0
stream counter=6454.0
stream counter=6455.0
stream counter=6456.0
stream counter=6457.0
stream counter=6458.0
stream counter=6459.0
stream counter=6460.0
stream counter=6461.0
stream counter=6462.0
...

-------------------------------------------
Time: 2022-02-25 08:28:52
-------------------------------------------
After 1350 samples: muhat=0.029528599284328876 varhat=1.0019304144292984



-------------------------------------------
Time: 2022-02-25 08:28:54
-------------------------------------------
stream counter=6622.0
stream counter=6623.0
stream counter=6624.0
stream counter=6625.0
stream counter=6626.0
stream counter=6627.0
stream counter=6628.0
stream counter=6629.0
stream counter=6630.0
stream counter=6631.0
...

-------------------------------------------
Time: 2022-02-25 08:28:54
-------------------------------------------
After 1517 samples: muhat=0.02484927360652866 varhat=1.0042114817919738



-------------------------------------------
Time: 2022-02-25 08:28:56
-------------------------------------------
stream counter=6789.0
stream counter=6790.0
stream counter=6791.0
stream counter=6792.0
stream counter=6793.0
stream counter=6794.0
stream counter=6795.0
stream counter=6796.0
stream counter=6797.0
stream counter=6798.0
...

-------------------------------------------
Time: 2022-02-25 08:28:56
-------------------------------------------
After 1685 samples: muhat=0.024258808605913333 varhat=1.0254995158349258



-------------------------------------------
Time: 2022-02-25 08:28:58
-------------------------------------------
stream counter=6957.0
stream counter=6958.0
stream counter=6959.0
stream counter=6960.0
stream counter=6961.0
stream counter=6962.0
stream counter=6963.0
stream counter=6964.0
stream counter=6965.0
stream counter=6966.0
...

-------------------------------------------
Time: 2022-02-25 08:28:58
-------------------------------------------
After 1853 samples: muhat=0.017491837032371248 varhat=1.0218049524874608



**N.B.**: With the dataLinearModel topic, mean estimation must converge to 0. Variance estimation must converge to 1.

In [115]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 08:28:59 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:3

## Recursive Least Squares (streaming - single model)

The following equations define a single step of the Recursive Least Squares (with forgetting factor $\nu$).

\begin{equation*}
\begin{cases}
V_{(t)}&=\frac{1}{\nu} \left(V_{(t-1)}
-\frac{V_{(t-1)} x^T_{t} x_{t} V_{(t-1)}}{1+ x_{t} V_{(t-1)} x^T_{t}} \right)\\[3pt]
\alpha_{(t)}&= V_{(t)} x^T_{t} \\[3pt]
e&= y_{t}- x_{t} \hat{\beta}_{(t-1)}  \\[3pt]
\hat{\beta}_{(t)}&=\hat{\beta}_{(t-1)}+ \alpha_{(t)} e \\[3pt]
\end{cases}
\end{equation*}

where $V$ is the covariance matrix and $\beta$ is the set of parameters of the linear model.
Details in the Streaming Analytics slides.


**Exercise**

The implementation of the state and an `updateFunction` to implement RLS in a streaming fashion is given below.

* Write the implementation of `RLSstep` function in order to perform the update of the state of the RLS model according to the equations given above.


In [116]:
def RLSstep(y,x,n,beta,V,nu):
    x.shape=(1,n)
    x1=np.append(1,x)
    x1.shape=(1,n+1)
    
    V=1.0/nu*(V-V.dot(x1.T).dot(x1).dot(V)/(1.0+float(x1.dot(V).dot(x1.T))))
    alpha=V.dot(x1.T)
    yhat=x1.dot(beta)
    err=y-yhat
    beta=beta+alpha*err
    
    return(beta,V,err,yhat)

In [117]:
n=10 # number of features
beta1=np.zeros(n+1)  ## initial parameter vector for model 1
v0=10 ## initialization covariance
V1=np.diag(np.zeros(n+1)+v0) ## initial covariance matrix for model 1
nu1=1.0 # forgetting factor for model 1

recentSize=10
D=np.zeros((recentSize,n+1)) #np.random.rand(recentSize,n+1) #np.zeros((20,n+1))+1
E=np.zeros((1,1))
mse=0
N=0
state1=(beta1,V1,nu1,mse,N,D,E)
initialStateRDD = sc.parallelize([('rls', state1)])

In [118]:
def updateFunction(new_values, state): 
    ## RLS update function
    ## Only update with first value of RDD.
    L=len(new_values)
    
    if (L>0):
        # Extract the data from state variable
        (beta,V,nu,sse,N,DD,E) = state
        #beta=state[0] ## Beta value
        #V=state[1]   ## V matrix
        #nu=state[2]   ## Forgetting factor
        #sse=state[3]  ## Sum of squared errors
        #N=state[4]   ## Number of treated samples
        #DD=state[5]  ## Analyzed values
        #E=state[6]   ## Error measure
        
        # Extract the values from the new_values variable
        yx=new_values[0] #Only using the first element in the batch
        i=yx[0] 
        y=yx[1]
        x=yx[2:]
        n=len(x)
        beta.shape=(n+1,1)
        
        # Compute RLS state update
        RL=RLSstep(y,x,n,beta,V,nu)
        
        # Update the state values using results from RLSstemp
        # Update beta and V values
        beta=RL[0]
        V=RL[1]
        E=np.append(E,RL[2])
        
        # Update sum of squares - for MSE computation
        sse=sse+pow(RL[2],2.0)
        
        # Append analyzed values to DD matrix
        d=yx[1:]
        d.shape=(1,n+1)
        DD=np.vstack((d,DD[:-1,:]))
        
        return (beta,V,nu,sse/(N+1.0),N+1,DD,E)  ## update formula mod1
        
    else:
        return state

In [120]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=2)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))

dataS=dataS.flatMap(lambda x: [('rls',1.0*np.array(x))])

updatedS=dataS.updateStateByKey(updateFunction,initialRDD=initialStateRDD)

## printing out updated values of the state
outbetaS=updatedS.map(lambda x: ": beta="+ np.array2string(x[1][0])).pprint()
outmseS=updatedS.map(lambda x: 'mse=' + str(x[1][3])).pprint()
outNS=updatedS.map(lambda x: x[1][4]).pprint()


In [121]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 08:29:22
-------------------------------------------
: beta=[[-0.38103405]
 [ 0.54682529]
 [ 0.22455623]
 [-0.01698979]
 [ 0.04960522]
 [-0.20081088]
 [ 0.27938294]
 [ 0.16902987]
 [ 0.04420558]
 [-0.00799865]
 [ 0.82649575]]

-------------------------------------------
Time: 2022-02-25 08:29:22
-------------------------------------------
mse=[[12.44075742]]

-------------------------------------------
Time: 2022-02-25 08:29:22
-------------------------------------------
1



-------------------------------------------
Time: 2022-02-25 08:29:24
-------------------------------------------
: beta=[[-0.40628075]
 [ 0.57467445]
 [ 0.18407214]
 [-0.02212698]
 [ 0.06331686]
 [-0.19375307]
 [ 0.28575722]
 [ 0.18001602]
 [ 0.00136048]
 [-0.00441446]
 [ 0.80605154]]

-------------------------------------------
Time: 2022-02-25 08:29:24
-------------------------------------------
mse=[[6.25134672]]

-------------------------------------------
Time: 2022-02-25 08:29:24
-------------------------------------------
2



-------------------------------------------
Time: 2022-02-25 08:29:26
-------------------------------------------
: beta=[[-0.3103209 ]
 [ 0.57753457]
 [ 0.21527254]
 [-0.12359252]
 [-0.06586684]
 [-0.14197148]
 [ 0.27665154]
 [ 0.22981476]
 [-0.11235202]
 [ 0.05209309]
 [ 0.85799434]]

-------------------------------------------
Time: 2022-02-25 08:29:26
-------------------------------------------
mse=[[2.25981575]]

-------------------------------------------
Time: 2022-02-25 08:29:26
-------------------------------------------
3



-------------------------------------------
Time: 2022-02-25 08:29:28
-------------------------------------------
: beta=[[-0.14526569]
 [ 0.67387141]
 [ 0.17366169]
 [-0.15277466]
 [-0.02715612]
 [-0.16320714]
 [ 0.25430558]
 [ 0.28091729]
 [-0.11508398]
 [-0.21768718]
 [ 0.86894035]]

-------------------------------------------
Time: 2022-02-25 08:29:28
-------------------------------------------
mse=[[0.78235997]]

-------------------------------------------
Time: 2022-02-25 08:29:28
-------------------------------------------
4



-------------------------------------------
Time: 2022-02-25 08:29:30
-------------------------------------------
: beta=[[-0.17599403]
 [ 0.70444766]
 [ 0.14818656]
 [-0.19501009]
 [-0.07975765]
 [-0.25093858]
 [ 0.15582316]
 [ 0.17745275]
 [-0.12514367]
 [-0.22720477]
 [ 0.87940588]]

-------------------------------------------
Time: 2022-02-25 08:29:30
-------------------------------------------
mse=[[0.17993572]]

-------------------------------------------
Time: 2022-02-25 08:29:30
-------------------------------------------
5



...

-------------------------------------------
Time: 2022-02-25 08:30:10
-------------------------------------------
: beta=[[ 5.47307616e-02]
 [ 9.89781970e-01]
 [-6.00250757e-03]
 [-5.15650399e-03]
 [-1.37935132e-02]
 [-2.10871867e-05]
 [-1.97679943e-02]
 [ 2.61708694e-03]
 [-3.00573953e-03]
 [ 2.86378189e-05]
 [ 9.95367146e-01]]

-------------------------------------------
Time: 2022-02-25 08:30:10
-------------------------------------------
mse=[[5.96783558e-05]]

-------------------------------------------
Time: 2022-02-25 08:30:10
-------------------------------------------
25



-------------------------------------------
Time: 2022-02-25 08:30:12
-------------------------------------------
: beta=[[ 5.48169638e-02]
 [ 9.90784913e-01]
 [-4.86257373e-03]
 [-7.06837289e-03]
 [-1.37894296e-02]
 [-8.57825806e-04]
 [-1.83802783e-02]
 [ 4.69067015e-03]
 [-1.76786501e-03]
 [ 5.76806958e-04]
 [ 9.95999834e-01]]

-------------------------------------------
Time: 2022-02-25 08:30:12
-------------------------------------------
mse=[[1.93208418e-05]]

-------------------------------------------
Time: 2022-02-25 08:30:12
-------------------------------------------
26



-------------------------------------------
Time: 2022-02-25 08:30:14
-------------------------------------------
: beta=[[ 5.24315673e-02]
 [ 9.89087172e-01]
 [-6.25494617e-03]
 [-4.65545939e-03]
 [-1.20397351e-02]
 [-9.32435842e-04]
 [-1.61107126e-02]
 [ 6.25878299e-03]
 [-7.03143028e-05]
 [ 1.79311081e-03]
 [ 9.95127700e-01]]

-------------------------------------------
Time: 2022-02-25 08:30:14
-------------------------------------------
mse=[[7.18156594e-05]]

-------------------------------------------
Time: 2022-02-25 08:30:14
-------------------------------------------
27



-------------------------------------------
Time: 2022-02-25 08:30:16
-------------------------------------------
: beta=[[ 5.21055058e-02]
 [ 9.89115723e-01]
 [-6.60326962e-03]
 [-4.35305485e-03]
 [-1.20460406e-02]
 [-9.05179300e-04]
 [-1.65806965e-02]
 [ 5.75744940e-03]
 [-2.27887588e-04]
 [ 1.67006593e-03]
 [ 9.95401214e-01]]

-------------------------------------------
Time: 2022-02-25 08:30:16
-------------------------------------------
mse=[[4.28200627e-06]]

-------------------------------------------
Time: 2022-02-25 08:30:16
-------------------------------------------
28



-------------------------------------------
Time: 2022-02-25 08:30:18
-------------------------------------------
: beta=[[ 5.21823412e-02]
 [ 9.89060772e-01]
 [-6.56509076e-03]
 [-4.45463260e-03]
 [-1.21815529e-02]
 [-1.01975960e-03]
 [-1.65907377e-02]
 [ 5.78950227e-03]
 [-1.82883145e-04]
 [ 1.67632979e-03]
 [ 9.95389209e-01]]

-------------------------------------------
Time: 2022-02-25 08:30:18
-------------------------------------------
mse=[[2.87397785e-07]]

-------------------------------------------
Time: 2022-02-25 08:30:18
-------------------------------------------
29



In [122]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 08:30:19 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:3

-------------------------------------------
Time: 2022-02-25 08:30:20
-------------------------------------------
: beta=[[ 5.49724710e-02]
 [ 9.94642033e-01]
 [-7.75004774e-03]
 [ 2.95648043e-04]
 [-1.04992797e-02]
 [ 2.84747271e-03]
 [-1.54998851e-02]
 [ 5.24171012e-03]
 [-2.44114076e-03]
 [-3.39041110e-03]
 [ 9.98345902e-01]]

-------------------------------------------
Time: 2022-02-25 08:30:20
-------------------------------------------
mse=[[0.00019586]]

-------------------------------------------
Time: 2022-02-25 08:30:20
-------------------------------------------
30



**Exercise**

If you look at the code above you will see that there is no guarantee that all samples are used. 
For instance, if the length `L` of the buffer in `new_values` is bigger than one (due to an excessive latency of the reception), the algorithm above will consider only the first vector `yx`. 

* How should the code be modified in order to consider all the elements of the buffer 'new_values'?

In [123]:
def RLSstep(y,x,n,beta,V,nu):
    x.shape=(1,n)
    x1=np.append(1,x)
    x1.shape=(1,n+1)
    
    V=1.0/nu*(V-V.dot(x1.T).dot(x1).dot(V)/(1.0+float(x1.dot(V).dot(x1.T))))
    alpha=V.dot(x1.T)
    yhat=x1.dot(beta)
    err=y-yhat
    beta=beta+alpha*err
    
    return(beta,V,err,yhat)

In [124]:
n=10 # number of features
beta1=np.zeros(n+1)  ## initial parameter vector for model 1
v0=10 ## initialization covariance
V1=np.diag(np.zeros(n+1)+v0) ## initial covariance matrix for model 1
nu1=1.0 # forgetting factor for model 1

recentSize=10
D=np.zeros((recentSize,n+1)) #np.random.rand(recentSize,n+1) #np.zeros((20,n+1))+1
E=np.zeros((1,1))
mse=0
N=0
state1=(beta1,V1,nu1,mse,N,D,E)
initialStateRDD = sc.parallelize([('rls', state1)])

In [125]:
def updateFunction(new_values, state): 
    ## RLS update function
    ## Only update with first value of RDD.
    L=len(new_values)
    
    if (L>0):
        # Extract the data from state variable
        (beta,V,nu,sse,N,DD,E) = state
        #beta=state[0] ## Beta value
        #V=state[1]   ## V matrix
        #nu=state[2]   ## Forgetting factor
        #sse=state[3]  ## Sum of squared errors
        #N=state[4]   ## Number of treated samples
        #DD=state[5]  ## Analyzed values
        #E=state[6]   ## Error measure
            
        # Extract the values from the new_values variable
        for yx in new_values: # Iterate over all the values in new values
            i=yx[0] 
            y=yx[1]
            x=yx[2:]
            n=len(x)
            beta.shape=(n+1,1)

            # Compute RLS state update
            RL=RLSstep(y,x,n,beta,V,nu)

            # Update the state values using results from RLSstemp
            # Update beta and V values
            beta=RL[0]
            V=RL[1]
            E=np.append(E,RL[2])

            # Update sum of squares - for MSE computation
            sse=sse+pow(RL[2],2.0)
            
            # Append analyzed values to DD matrix
            d=yx[1:]
            d.shape=(1,n+1)
            DD=np.vstack((d,DD[:-1,:]))
            
            #Update the value of N
            N+=1.0
        
        return (beta,V,nu,sse/N,N,DD,E)  ## update formula mod1
        
    else:
        return state

In [126]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=2)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))

dataS=dataS.flatMap(lambda x: [('rls',1.0*np.array(x))])

updatedS=dataS.updateStateByKey(updateFunction,initialRDD=initialStateRDD)

## printing out updated values of the state
outbetaS=updatedS.map(lambda x: ": beta="+ np.array2string(x[1][0])).pprint()
outmseS=updatedS.map(lambda x: 'mse=' + str(x[1][3])).pprint()
outNS=updatedS.map(lambda x: x[1][4]).pprint()

In [127]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 08:31:06
-------------------------------------------
: beta=[[ 0.05228105]
 [ 0.9946191 ]
 [ 0.00170674]
 [ 0.00462351]
 [-0.00111272]
 [ 0.0017683 ]
 [ 0.00390102]
 [-0.0023184 ]
 [-0.00205778]
 [ 0.00128027]
 [ 1.00493732]]

-------------------------------------------
Time: 2022-02-25 08:31:06
-------------------------------------------
mse=[[0.11194501]]

-------------------------------------------
Time: 2022-02-25 08:31:06
-------------------------------------------
94.0



-------------------------------------------
Time: 2022-02-25 08:31:08
-------------------------------------------
: beta=[[ 5.16555031e-02]
 [ 9.98502685e-01]
 [-4.88884534e-04]
 [ 1.05739216e-03]
 [-1.76751508e-03]
 [-4.41619492e-04]
 [ 2.02392161e-03]
 [-1.79978790e-03]
 [-5.61587083e-04]
 [ 3.47565408e-04]
 [ 1.00114375e+00]]

-------------------------------------------
Time: 2022-02-25 08:31:08
-------------------------------------------
mse=[[0.00102969]]

-------------------------------------------
Time: 2022-02-25 08:31:08
-------------------------------------------
266.0



-------------------------------------------
Time: 2022-02-25 08:31:10
-------------------------------------------
: beta=[[ 4.90797996e-02]
 [ 9.99260984e-01]
 [-1.81597427e-03]
 [ 2.25811915e-03]
 [-1.86882086e-03]
 [-1.30129117e-04]
 [ 8.07119978e-04]
 [-1.75558120e-03]
 [-8.04239102e-04]
 [ 6.75977064e-04]
 [ 9.99780956e-01]]

-------------------------------------------
Time: 2022-02-25 08:31:10
-------------------------------------------
mse=[[0.00032267]]

-------------------------------------------
Time: 2022-02-25 08:31:10
-------------------------------------------
438.0



-------------------------------------------
Time: 2022-02-25 08:31:12
-------------------------------------------
: beta=[[ 4.98123725e-02]
 [ 1.00023260e+00]
 [-2.55219600e-03]
 [ 2.33985506e-03]
 [-1.38673153e-03]
 [ 4.92351862e-04]
 [ 6.25674448e-04]
 [-1.41933050e-04]
 [-2.08040239e-04]
 [ 1.20557952e-03]
 [ 9.99262060e-01]]

-------------------------------------------
Time: 2022-02-25 08:31:12
-------------------------------------------
mse=[[0.00022459]]

-------------------------------------------
Time: 2022-02-25 08:31:12
-------------------------------------------
609.0



-------------------------------------------
Time: 2022-02-25 08:31:14
-------------------------------------------
: beta=[[ 5.01371755e-02]
 [ 1.00032374e+00]
 [-2.27501032e-03]
 [ 1.40790519e-03]
 [-1.34461752e-03]
 [ 9.98909277e-04]
 [ 7.29586206e-04]
 [-8.44833488e-04]
 [-4.77051663e-04]
 [ 7.86012433e-04]
 [ 9.99260248e-01]]

-------------------------------------------
Time: 2022-02-25 08:31:14
-------------------------------------------
mse=[[0.00016755]]

-------------------------------------------
Time: 2022-02-25 08:31:14
-------------------------------------------
778.0



In [128]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 08:31:15 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:3

## Recursive Least Squares (streaming - multiple models in parallel)

The code in the previous section can be extended to support multiple models running in parallel.

* The *state* is modified in order to include the states of the two different models.
    Each state contains a state of variables to keep the state of the model, as well as to keep track of MSE estimates. 
    A state is a list of 5 elements:
    * The first three are beta, V and mu, and define the state of the model (see RLS formulas in course)
    * The last two are an estimate of the MSE of the model, and the number of treated samples
   The two states are combined in a key-value RDD in the following way:
   ```python
    initialStateRDD = sc.parallelize([(u'rls1', state1),(u'rls2', state2)])
   ```
   with `rlsX` being the key and the value being the corresponding state

* The original Dstream is duplicated using a `flatMap` operation in order to create two key-pair streams, containing the key corresponding to the model, as well as a copy of the data in the original Dstream (`dataS` here).
```python
dataS2=dataS.flatMap(lambda x: [('rls1',('rls1',1.0*np.array(x))),
                            ('rls2',('rls2',1.0*np.array(x)))])
```

* The *update function* is modified to handle the new format of the stream (by means of the `key` variable), by storing this value from the incoming state and re-transmitting it in the updated state.

* Note that there is only a single `updateFunction`, since the `updateStateByKey` operation applies the update function in parallel to all the different keys (in this case, each key correspond to a model).


In [129]:
def RLSstep(y,x,n,beta,V,nu):
    x.shape=(1,n)
    x1=np.append(1,x)
    x1.shape=(1,n+1)
    
    V=1.0/nu*(V-V.dot(x1.T).dot(x1).dot(V)/(1.0+float(x1.dot(V).dot(x1.T))))
    alpha=V.dot(x1.T)
    yhat=x1.dot(beta)
    err=y-yhat
    beta=beta+alpha*err
    
    return(beta,V,err,yhat)

In [130]:
n=10 # number of features
beta1=np.zeros(n+1)  ## initial parameter vector for model 1
v0=10 ## initialization covariance
V1=np.diag(np.zeros(n+1)+v0) ## initial covariance matrix for model 1
nu1=1.0 # forgetting factor for model 1

batchSize=10
D=np.zeros((batchSize,n+1)) ## this is not used here 
E=np.zeros((1,1))
mse=0
N=0
state1=('mod1',beta1,V1,nu1,mse,N,D,E,0,0)

nu2=0.99 # forgetting factor for model 2

## Note that the only difference is the forgetting value
state2=('mod2',beta1,V1,nu2,mse,N,D,E,0,0)

initialStateRDD = sc.parallelize([(u'rls1', state1),
                                  (u'rls2', state2)])

In [131]:
def updateFunction(new_values, state): 
    ## RLS update function
    ## Only update with first value of RDD. 
    L=len(new_values)  ## size of the buffer
    if (L>0):
        # Extract the data from state variable
        (model_key,beta,V,nu,sse,N,DD,E,y,yhat) = state
        #model_key=state[0] ## Unique identifier for the model
        #beta=state[1] ## Beta value
        #V=state[2]   ## V matrix
        #nu=state[3]   ## Forgetting factor
        #sse=state[4]  ## Sum of squared errors
        #N=state[5]   ## Number of treated samples
        #DD=state[6]  ## Analyzed values
        #E=state[7]   ## Error measure
        #y=state[8]   ## True value for the last value in the batch
        #yhat=state[9] ## Predicted value for the last value in the batch
        
        # Extract data from the new_values vector
        # new_values = (key,RDD([counter,y,x]))
        key=new_values[0][0]
        yx=new_values[0][1]   ### this is only the first value in the buffer
        i=yx[0]
        y=yx[1]
        x=yx[2:]
        n=len(x)
        beta.shape=(n+1,1)
        
        # Compute RLS state update
        RL=RLSstep(y,x,n,beta,V,nu)
        
        # Update the state values using results from RLSstep
        beta=RL[0]
        V=RL[1]
        err=RL[2]
        E=np.append(E,err)
        yhat=RL[3]
        
        # Update sum of squares - for MSE computation
        sse=sse+pow(err,2.0)
         
        # Append analyzed values to DD matrix
        d=yx[1:]
        d.shape=(1,n+1)
        DD=np.vstack((d,DD[:-1,:]))
        
        return (key,beta,V,nu,sse/(N+1.0),N+1,DD,E,y,yhat)  
        
    else:
        return state

In [133]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=2)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))

dataS.map(lambda x: "stream counter="+ str(x[0])).pprint()

dataS2=dataS.flatMap(lambda x: [('rls1',('rls1',1.0*np.array(x))),
                            ('rls2',('rls2',1.0*np.array(x)))])

updatedS=dataS2.updateStateByKey(updateFunction,initialRDD=initialStateRDD)#.pprint()

## printing out updated values of the states of the two models
outbetaS=updatedS.map(lambda x: x[1][0]+": beta="+ np.array2string(x[1][1])).pprint()
outmseS=updatedS.map(lambda x: x[1][0]+": mse="+ np.array2string(x[1][4])).pprint()
outNS=updatedS.map(lambda x: x[1][0]+": N="+ str(x[1][5])).pprint()
outyS=updatedS.map(lambda x: x[1][0]+": y="+ str(x[1][8])+ " yhat="+ str(x[1][9][0])).pprint()


In [134]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 08:31:54
-------------------------------------------
stream counter=13133.0
stream counter=13134.0
stream counter=13135.0
stream counter=13136.0
stream counter=13137.0
stream counter=13138.0
stream counter=13139.0
stream counter=13140.0
stream counter=13141.0
stream counter=13142.0
...

-------------------------------------------
Time: 2022-02-25 08:31:54
-------------------------------------------
rls1: beta=[[ 0.11578906]
 [ 0.19450314]
 [-0.22029551]
 [-0.20833767]
 [ 0.21558564]
 [ 0.16094819]
 [ 0.00651076]
 [-0.18300858]
 [-0.10137686]
 [ 0.16517338]
 [ 0.10384654]]
rls2: beta=[[ 0.11695865]
 [ 0.19646781]
 [-0.22252072]
 [-0.21044209]
 [ 0.21776327]
 [ 0.16257393]
 [ 0.00657652]
 [-0.18485715]
 [-0.10240086]
 [ 0.1668418 ]
 [ 0.1048955 ]]

-------------------------------------------
Time: 2022-02-25 08:31:54
-------------------------------------------
rls1: mse=[[6.65808614]]
rls2: mse=[[6.65808614]]

-----------------

-------------------------------------------
Time: 2022-02-25 08:31:56
-------------------------------------------
stream counter=13260.0
stream counter=13261.0
stream counter=13262.0
stream counter=13263.0
stream counter=13264.0
stream counter=13265.0
stream counter=13266.0
stream counter=13267.0
stream counter=13268.0
stream counter=13269.0
...

-------------------------------------------
Time: 2022-02-25 08:31:56
-------------------------------------------
rls1: beta=[[-0.03885705]
 [ 0.44158788]
 [ 0.0112992 ]
 [-0.0873766 ]
 [ 0.40739751]
 [ 0.03149053]
 [-0.17170338]
 [-0.39023288]
 [-0.0223828 ]
 [ 0.09804632]
 [ 0.17609965]]
rls2: beta=[[-0.03971246]
 [ 0.44678798]
 [ 0.01210658]
 [-0.08789711]
 [ 0.41208681]
 [ 0.0314211 ]
 [-0.17397122]
 [-0.39479493]
 [-0.02237243]
 [ 0.09883575]
 [ 0.17809472]]

-------------------------------------------
Time: 2022-02-25 08:31:56
-------------------------------------------
rls1: mse=[[4.61791429]]
rls2: mse=[[4.62526178]]

-----------------

-------------------------------------------
Time: 2022-02-25 08:31:58
-------------------------------------------
stream counter=13430.0
stream counter=13431.0
stream counter=13432.0
stream counter=13433.0
stream counter=13434.0
stream counter=13435.0
stream counter=13436.0
stream counter=13437.0
stream counter=13438.0
stream counter=13439.0
...

-------------------------------------------
Time: 2022-02-25 08:31:58
-------------------------------------------
rls1: beta=[[-0.13650766]
 [ 0.45293953]
 [ 0.08098542]
 [-0.1171693 ]
 [ 0.49294328]
 [ 0.17227222]
 [-0.14034034]
 [-0.24398119]
 [ 0.00235458]
 [ 0.09535816]
 [ 0.24080297]]
rls2: beta=[[-0.13875534]
 [ 0.45830518]
 [ 0.08278832]
 [-0.11811186]
 [ 0.49885317]
 [ 0.1742043 ]
 [-0.14216471]
 [-0.24646513]
 [ 0.00271829]
 [ 0.09610825]
 [ 0.24372006]]

-------------------------------------------
Time: 2022-02-25 08:31:58
-------------------------------------------
rls1: mse=[[1.910601]]
rls2: mse=[[1.91599246]]

-------------------

-------------------------------------------
Time: 2022-02-25 08:32:00
-------------------------------------------
stream counter=13602.0
stream counter=13603.0
stream counter=13604.0
stream counter=13605.0
stream counter=13606.0
stream counter=13607.0
stream counter=13608.0
stream counter=13609.0
stream counter=13610.0
stream counter=13611.0
...

-------------------------------------------
Time: 2022-02-25 08:32:00
-------------------------------------------
rls1: beta=[[-0.13386396]
 [ 0.44329225]
 [ 0.08396221]
 [-0.12125291]
 [ 0.49329359]
 [ 0.17182961]
 [-0.14584127]
 [-0.24697087]
 [ 0.00334094]
 [ 0.09492166]
 [ 0.2507264 ]]
rls2: beta=[[-0.13553047]
 [ 0.44653533]
 [ 0.08642092]
 [-0.12309403]
 [ 0.49928124]
 [ 0.17366452]
 [-0.14887648]
 [-0.25011266]
 [ 0.00392196]
 [ 0.09557554]
 [ 0.25582773]]

-------------------------------------------
Time: 2022-02-25 08:32:00
-------------------------------------------
rls1: mse=[[0.47814059]]
rls2: mse=[[0.47971287]]

-----------------

-------------------------------------------
Time: 2022-02-25 08:32:02
-------------------------------------------
stream counter=13775.0
stream counter=13776.0
stream counter=13777.0
stream counter=13778.0
stream counter=13779.0
stream counter=13780.0
stream counter=13781.0
stream counter=13782.0
stream counter=13783.0
stream counter=13784.0
...

-------------------------------------------
Time: 2022-02-25 08:32:02
-------------------------------------------
rls1: beta=[[-0.01243835]
 [ 0.54964569]
 [ 0.07304174]
 [-0.19206198]
 [ 0.52257145]
 [ 0.00509566]
 [-0.26786578]
 [-0.02496995]
 [-0.03808205]
 [ 0.11101498]
 [ 0.27826978]]
rls2: beta=[[-8.82174166e-03]
 [ 5.57435376e-01]
 [ 7.50982785e-02]
 [-1.97029441e-01]
 [ 5.29876393e-01]
 [-3.19624417e-04]
 [-2.76291792e-01]
 [-1.84665761e-02]
 [-3.92803359e-02]
 [ 1.12357328e-01]
 [ 2.84708635e-01]]

-------------------------------------------
Time: 2022-02-25 08:32:02
-------------------------------------------
rls1: mse=[[0.19507398]]

...

-------------------------------------------
Time: 2022-02-25 08:32:58
-------------------------------------------
stream counter=18630.0
stream counter=18631.0
stream counter=18632.0
stream counter=18633.0
stream counter=18634.0
stream counter=18635.0
stream counter=18636.0
stream counter=18637.0
stream counter=18638.0
stream counter=18639.0
...

-------------------------------------------
Time: 2022-02-25 08:32:58
-------------------------------------------
rls1: beta=[[ 5.61522722e-02]
 [ 9.89218924e-01]
 [-5.16622909e-04]
 [-4.37513752e-04]
 [-3.70647911e-03]
 [-7.99955917e-03]
 [ 1.28628698e-02]
 [ 1.21921047e-02]
 [-3.24868779e-04]
 [ 3.04755180e-03]
 [ 9.93633093e-01]]
rls2: beta=[[ 5.58965814e-02]
 [ 9.92050592e-01]
 [-2.21339277e-03]
 [-8.90852167e-04]
 [-1.08153755e-03]
 [-7.92747380e-03]
 [ 1.16509241e-02]
 [ 1.20966535e-02]
 [ 1.40734833e-04]
 [ 1.69845102e-03]
 [ 9.95898953e-01]]

-------------------------------------------
Time: 2022-02-25 08:32:58
------------------------

-------------------------------------------
Time: 2022-02-25 08:33:00
-------------------------------------------
stream counter=18803.0
stream counter=18804.0
stream counter=18805.0
stream counter=18806.0
stream counter=18807.0
stream counter=18808.0
stream counter=18809.0
stream counter=18810.0
stream counter=18811.0
stream counter=18812.0
...

-------------------------------------------
Time: 2022-02-25 08:33:00
-------------------------------------------
rls1: beta=[[ 5.67650067e-02]
 [ 9.89154167e-01]
 [-5.51778925e-04]
 [-3.72526241e-05]
 [-3.56791964e-03]
 [-7.35047430e-03]
 [ 1.25242338e-02]
 [ 1.21378038e-02]
 [-7.05636383e-04]
 [ 2.83526172e-03]
 [ 9.93620894e-01]]
rls2: beta=[[ 5.64627785e-02]
 [ 9.91993274e-01]
 [-2.26237780e-03]
 [-5.39050164e-04]
 [-9.46229908e-04]
 [-7.32073515e-03]
 [ 1.13138763e-02]
 [ 1.20281514e-02]
 [-1.66624323e-04]
 [ 1.50201426e-03]
 [ 9.95876410e-01]]

-------------------------------------------
Time: 2022-02-25 08:33:00
------------------------

-------------------------------------------
Time: 2022-02-25 08:33:02
-------------------------------------------
stream counter=18975.0
stream counter=18976.0
stream counter=18977.0
stream counter=18978.0
stream counter=18979.0
stream counter=18980.0
stream counter=18981.0
stream counter=18982.0
stream counter=18983.0
stream counter=18984.0
...

-------------------------------------------
Time: 2022-02-25 08:33:02
-------------------------------------------
rls1: beta=[[ 5.68780588e-02]
 [ 9.89089693e-01]
 [-9.18326116e-04]
 [-4.21973296e-04]
 [-3.28541199e-03]
 [-7.51314080e-03]
 [ 1.26745720e-02]
 [ 1.23849962e-02]
 [-8.31075073e-04]
 [ 2.51871483e-03]
 [ 9.93919069e-01]]
rls2: beta=[[ 5.64365714e-02]
 [ 9.92010789e-01]
 [-2.16854161e-03]
 [-4.33649852e-04]
 [-1.02159477e-03]
 [-7.27921804e-03]
 [ 1.12754645e-02]
 [ 1.19645802e-02]
 [-1.34662650e-04]
 [ 1.57844205e-03]
 [ 9.95801660e-01]]

-------------------------------------------
Time: 2022-02-25 08:33:02
------------------------

-------------------------------------------
Time: 2022-02-25 08:33:04
-------------------------------------------
stream counter=19146.0
stream counter=19147.0
stream counter=19148.0
stream counter=19149.0
stream counter=19150.0
stream counter=19151.0
stream counter=19152.0
stream counter=19153.0
stream counter=19154.0
stream counter=19155.0
...

-------------------------------------------
Time: 2022-02-25 08:33:04
-------------------------------------------
rls1: beta=[[ 5.70876404e-02]
 [ 9.89279785e-01]
 [-9.90029802e-04]
 [-1.13961554e-04]
 [-3.37531572e-03]
 [-7.26780638e-03]
 [ 1.25627866e-02]
 [ 1.21628296e-02]
 [-6.87909137e-04]
 [ 2.63995925e-03]
 [ 9.93770761e-01]]
rls2: beta=[[ 5.66153388e-02]
 [ 9.92168439e-01]
 [-2.23330888e-03]
 [-1.85585489e-04]
 [-1.08401460e-03]
 [-7.07843782e-03]
 [ 1.11679987e-02]
 [ 1.17675674e-02]
 [-1.36216639e-05]
 [ 1.67819578e-03]
 [ 9.95663675e-01]]

-------------------------------------------
Time: 2022-02-25 08:33:04
------------------------

-------------------------------------------
Time: 2022-02-25 08:33:06
-------------------------------------------
stream counter=19320.0
stream counter=19321.0
stream counter=19322.0
stream counter=19323.0
stream counter=19324.0
stream counter=19325.0
stream counter=19326.0
stream counter=19327.0
stream counter=19328.0
stream counter=19329.0
...

-------------------------------------------
Time: 2022-02-25 08:33:06
-------------------------------------------
rls1: beta=[[ 5.68805601e-02]
 [ 9.88142931e-01]
 [-2.28663664e-03]
 [-2.53572078e-04]
 [-1.57598886e-03]
 [-7.21827158e-03]
 [ 1.18209673e-02]
 [ 1.36356195e-02]
 [ 1.01413374e-03]
 [ 2.38926345e-03]
 [ 9.93768942e-01]]
rls2: beta=[[ 5.64254334e-02]
 [ 9.90871406e-01]
 [-3.70416197e-03]
 [-3.34275237e-04]
 [ 8.67067686e-04]
 [-6.93226867e-03]
 [ 1.03088680e-02]
 [ 1.34680687e-02]
 [ 1.90072639e-03]
 [ 1.40247461e-03]
 [ 9.95728454e-01]]

-------------------------------------------
Time: 2022-02-25 08:33:06
------------------------

In [135]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 08:33:08 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


-------------------------------------------
Time: 2022-02-25 08:33:08
-------------------------------------------
stream counter=19493.0
stream counter=19494.0
stream counter=19495.0
stream counter=19496.0
stream counter=19497.0
stream counter=19498.0
stream counter=19499.0
stream counter=19500.0
stream counter=19501.0
stream counter=19502.0
...

-------------------------------------------
Time: 2022-02-25 08:33:08
-------------------------------------------
rls1: beta=[[ 5.76677445e-02]
 [ 9.87819207e-01]
 [-2.20358377e-03]
 [-1.31142572e-03]
 [-2.24192468e-03]
 [-6.91996593e-03]
 [ 1.05570531e-02]
 [ 1.40175551e-02]
 [-3.98729093e-04]
 [ 3.18925196e-03]
 [ 9.94274021e-01]]
rls2: beta=[[ 5.73652696e-02]
 [ 9.90454405e-01]
 [-3.68438068e-03]
 [-1.55833058e-03]
 [ 2.01049072e-04]
 [-6.56744756e-03]
 [ 8.83515021e-03]
 [ 1.38588010e-02]
 [ 4.11328514e-04]
 [ 2.26388408e-03]
 [ 9.96169626e-01]]

-------------------------------------------
Time: 2022-02-25 08:33:08
------------------------

Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurren

**Exercise**

If you look at the code above you will see that there is no guarantee that all samples are used. 
For instance, if the length `L` of the buffer in `new_values` is bigger than one (due to an excessive latency of the reception), the algorithm above will consider only the first vector `yx`. 

* How should the code be modified in order to consider all the element of the buffer 'new_values'?

In [11]:
def RLSstep(y,x,n,beta,V,nu):
    x.shape=(1,n)
    x1=np.append(1,x)
    x1.shape=(1,n+1)
    
    V=1.0/nu*(V-V.dot(x1.T).dot(x1).dot(V)/(1.0+float(x1.dot(V).dot(x1.T))))
    alpha=V.dot(x1.T)
    yhat=x1.dot(beta)
    err=y-yhat
    beta=beta+alpha*err
    
    return(beta,V,err,yhat)

In [12]:
n=10 # number of features
beta1=np.zeros(n+1)  ## initial parameter vector for model 1
v0=10 ## initialization covariance
V1=np.diag(np.zeros(n+1)+v0) ## initial covariance matrix for model 1
nu1=1.0 # forgetting factor for model 1

batchSize=10
D=np.zeros((batchSize,n+1)) ## this is not used here 
E=np.zeros((1,1))
mse=0
N=0
state1=('mod1',beta1,V1,nu1,mse,N,D,E,0,0)

nu2=0.99 # forgetting factor for model 2

## Note that the only difference is the forgetting value
state2=('mod2',beta1,V1,nu2,mse,N,D,E,0,0)

initialStateRDD = sc.parallelize([(u'rls1', state1),
                                  (u'rls2', state2)])

In [13]:
def updateFunction(new_values, state): 
    ## RLS update function
    ## Only update with first value of RDD. 
    L=len(new_values)  ## size of the buffer
    if (L>0):
        # Extract the data from state variable
        (model_key,beta,V,nu,sse,N,DD,E,y,yhat) = state
        #model_key=state[0] ## Unique identifier for the model
        #beta=state[1] ## Beta value
        #V=state[2]   ## V matrix
        #nu=state[3]   ## Forgetting factor
        #sse=state[4]  ## Sum of squared errors
        #N=state[5]   ## Number of treated samples
        #DD=state[6]  ## Analyzed values
        #E=state[7]   ## Error measure
        #y=state[8]   ## True value for the last value in the batch
        #yhat=state[9] ## Predicted value for the last value in the batch
        
        # Extract data from the new_values vector
        # new_values = (key,RDD([counter,y,x]))
        for curr_value in new_values: # Iterate over all the values in new_values
            key=curr_value[0]
            yx=curr_value[1]   ### this is only the first value in the buffer
            i=yx[0]
            y=yx[1]
            x=yx[2:]
            n=len(x)
            beta.shape=(n+1,1)

            # Compute RLS state update
            RL=RLSstep(y,x,n,beta,V,nu)

            # Update the state values using results from RLSstep
            beta=RL[0]
            V=RL[1]
            err=RL[2]
            E=np.append(E,err)
            yhat=RL[3]

            # Update sum of squares - for MSE computation
            sse=sse+pow(err,2.0)
            
            # Append analyzed values to DD matrix
            d=yx[1:]
            d.shape=(1,n+1)
            DD=np.vstack((d,DD[:-1,:]))
            
            #Update the value of N
            N+=1.0
            
        return (key,beta,V,nu,sse/N,N,DD,E,y,yhat)  
        
    else:
        return state

In [14]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=2)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))

dataS.map(lambda x: "stream counter="+ str(x[0])).pprint()

dataS2=dataS.flatMap(lambda x: [('rls1',('rls1',1.0*np.array(x))),
                            ('rls2',('rls2',1.0*np.array(x)))])

updatedS=dataS2.updateStateByKey(updateFunction,initialRDD=initialStateRDD)#.pprint()

## printing out updated values of the states of the two models
outbetaS=updatedS.map(lambda x: x[1][0]+": beta="+ np.array2string(x[1][1])).pprint()
outmseS=updatedS.map(lambda x: x[1][0]+": mse="+ np.array2string(x[1][4])).pprint()
outNS=updatedS.map(lambda x: x[1][0]+": N="+ str(x[1][5])).pprint()
outyS=updatedS.map(lambda x: x[1][0]+": y="+ str(x[1][8])+ " yhat="+ str(x[1][9][0])).pprint()


In [15]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 08:36:52
-------------------------------------------
stream counter=26690.0
stream counter=26691.0
stream counter=26692.0
stream counter=26693.0
stream counter=26694.0
stream counter=26695.0
stream counter=26696.0
stream counter=26697.0
stream counter=26698.0
stream counter=26699.0

-------------------------------------------
Time: 2022-02-25 08:36:52
-------------------------------------------
rls1: beta=[[ 0.13028591]
 [ 0.61921339]
 [ 0.15924526]
 [ 0.03849573]
 [ 0.2737287 ]
 [-0.15882164]
 [-0.00795231]
 [ 0.03966675]
 [-0.12010643]
 [ 0.45549448]
 [ 0.7397986 ]]
rls2: beta=[[ 0.14020291]
 [ 0.6270797 ]
 [ 0.16149946]
 [ 0.04844615]
 [ 0.27495878]
 [-0.15275818]
 [-0.00475571]
 [ 0.04171117]
 [-0.11971959]
 [ 0.46203791]
 [ 0.74880343]]

-------------------------------------------
Time: 2022-02-25 08:36:52
-------------------------------------------
rls1: mse=[[0.71298455]]
rls2: mse=[[0.71267187]]

---------------------

-------------------------------------------
Time: 2022-02-25 08:36:54
-------------------------------------------
stream counter=26700.0
stream counter=26701.0
stream counter=26702.0
stream counter=26703.0
stream counter=26704.0
stream counter=26705.0
stream counter=26706.0
stream counter=26707.0
stream counter=26708.0
stream counter=26709.0
...

-------------------------------------------
Time: 2022-02-25 08:36:54
-------------------------------------------
rls1: beta=[[ 4.93074155e-02]
 [ 9.99750583e-01]
 [-2.11424973e-03]
 [-4.45648298e-04]
 [-2.89482271e-03]
 [ 1.73210089e-03]
 [ 3.33074546e-04]
 [-3.26872294e-04]
 [ 2.37188129e-03]
 [-2.02204330e-03]
 [ 1.00068194e+00]]
rls2: beta=[[ 4.73771113e-02]
 [ 1.00099117e+00]
 [-4.12823759e-03]
 [-1.19783035e-03]
 [-2.68595195e-03]
 [ 2.17663857e-03]
 [ 6.60979874e-04]
 [ 6.15016991e-04]
 [ 3.23861153e-03]
 [-2.34744675e-03]
 [ 1.00182962e+00]]

-------------------------------------------
Time: 2022-02-25 08:36:54
------------------------

-------------------------------------------
Time: 2022-02-25 08:36:56
-------------------------------------------
stream counter=26880.0
stream counter=26881.0
stream counter=26882.0
stream counter=26883.0
stream counter=26884.0
stream counter=26885.0
stream counter=26886.0
stream counter=26887.0
stream counter=26888.0
stream counter=26889.0
...

-------------------------------------------
Time: 2022-02-25 08:36:56
-------------------------------------------
rls1: beta=[[ 4.95383987e-02]
 [ 9.98953467e-01]
 [-9.51299257e-04]
 [ 1.03247914e-03]
 [ 1.00903159e-03]
 [ 1.91929685e-03]
 [-8.69097571e-04]
 [-1.27355183e-03]
 [ 2.49606042e-03]
 [-1.92001135e-03]
 [ 1.00177169e+00]]
rls2: beta=[[ 5.06149155e-02]
 [ 9.98332539e-01]
 [-9.10393969e-04]
 [ 2.66008437e-03]
 [ 3.02960425e-03]
 [ 4.02431103e-03]
 [-9.31114231e-05]
 [-1.29123076e-03]
 [ 5.45570361e-04]
 [-2.64400586e-03]
 [ 1.00329977e+00]]

-------------------------------------------
Time: 2022-02-25 08:36:56
------------------------

-------------------------------------------
Time: 2022-02-25 08:36:58
-------------------------------------------
stream counter=27058.0
stream counter=27059.0
stream counter=27060.0
stream counter=27061.0
stream counter=27062.0
stream counter=27063.0
stream counter=27064.0
stream counter=27065.0
stream counter=27066.0
stream counter=27067.0
...

-------------------------------------------
Time: 2022-02-25 08:36:58
-------------------------------------------
rls1: beta=[[ 5.01316327e-02]
 [ 9.99814229e-01]
 [ 7.23765372e-04]
 [ 9.44587425e-04]
 [ 3.14655763e-03]
 [ 1.24590906e-03]
 [-3.88915095e-04]
 [-4.01435741e-05]
 [ 1.88437249e-04]
 [-6.36431443e-04]
 [ 1.00152144e+00]]
rls2: beta=[[ 5.30207138e-02]
 [ 1.00273328e+00]
 [ 3.89458626e-03]
 [ 9.78036331e-04]
 [ 6.41634081e-03]
 [-2.78604525e-04]
 [ 1.87021536e-03]
 [ 1.07885074e-03]
 [-4.62372635e-03]
 [ 2.33755661e-03]
 [ 1.00189942e+00]]

-------------------------------------------
Time: 2022-02-25 08:36:58
------------------------

-------------------------------------------
Time: 2022-02-25 08:37:00
-------------------------------------------
stream counter=27231.0
stream counter=27232.0
stream counter=27233.0
stream counter=27234.0
stream counter=27235.0
stream counter=27236.0
stream counter=27237.0
stream counter=27238.0
stream counter=27239.0
stream counter=27240.0
...

-------------------------------------------
Time: 2022-02-25 08:37:00
-------------------------------------------
rls1: beta=[[ 4.97389406e-02]
 [ 1.00008832e+00]
 [ 1.54506436e-04]
 [ 1.00765898e-03]
 [ 1.97069565e-03]
 [ 1.75327937e-03]
 [-1.70645872e-04]
 [ 3.89461248e-04]
 [ 6.21851365e-04]
 [ 4.18087757e-04]
 [ 1.00112818e+00]]
rls2: beta=[[ 5.02019562e-02]
 [ 1.00276279e+00]
 [-1.85975190e-03]
 [ 2.10917791e-03]
 [-1.24695823e-05]
 [ 2.88133569e-03]
 [ 2.06932116e-03]
 [ 1.76740922e-03]
 [ 1.42097450e-03]
 [ 3.10131217e-03]
 [ 9.98555893e-01]]

-------------------------------------------
Time: 2022-02-25 08:37:00
------------------------

...

-------------------------------------------
Time: 2022-02-25 08:38:24
-------------------------------------------
stream counter=34558.0
stream counter=34559.0
stream counter=34560.0
stream counter=34561.0
stream counter=34562.0
stream counter=34563.0
stream counter=34564.0
stream counter=34565.0
stream counter=34566.0
stream counter=34567.0
...

-------------------------------------------
Time: 2022-02-25 08:38:24
-------------------------------------------
rls1: beta=[[ 4.98086292e-02]
 [ 1.00033713e+00]
 [-6.54550515e-04]
 [ 4.90403770e-04]
 [ 2.07278612e-04]
 [ 3.26308221e-05]
 [ 1.68696807e-04]
 [-2.89562694e-06]
 [-1.23089832e-04]
 [ 4.06597840e-04]
 [ 1.00014874e+00]]
rls2: beta=[[ 5.05003823e-02]
 [ 1.00105893e+00]
 [-4.88295271e-04]
 [-2.96516357e-03]
 [-2.46712578e-03]
 [ 3.28933874e-03]
 [ 3.96099949e-03]
 [-1.08211609e-03]
 [-1.94862018e-03]
 [ 3.26391299e-04]
 [ 9.97377362e-01]]

-------------------------------------------
Time: 2022-02-25 08:38:24
------------------------

-------------------------------------------
Time: 2022-02-25 08:38:26
-------------------------------------------
stream counter=34731.0
stream counter=34732.0
stream counter=34733.0
stream counter=34734.0
stream counter=34735.0
stream counter=34736.0
stream counter=34737.0
stream counter=34738.0
stream counter=34739.0
stream counter=34740.0
...

-------------------------------------------
Time: 2022-02-25 08:38:26
-------------------------------------------
rls1: beta=[[ 4.97435638e-02]
 [ 1.00033638e+00]
 [-7.05019967e-04]
 [ 4.24953998e-04]
 [ 2.12840219e-04]
 [ 1.20615251e-04]
 [ 2.35317482e-04]
 [ 9.79187904e-07]
 [-1.36476954e-04]
 [ 4.54199920e-04]
 [ 1.00008369e+00]]
rls2: beta=[[ 4.79727981e-02]
 [ 1.00121834e+00]
 [-3.60406596e-03]
 [-2.61391237e-03]
 [-7.29605068e-04]
 [ 3.44168840e-03]
 [ 4.37644554e-03]
 [ 8.38812305e-04]
 [ 3.43356800e-05]
 [ 2.25856266e-03]
 [ 9.95408574e-01]]

-------------------------------------------
Time: 2022-02-25 08:38:26
------------------------

-------------------------------------------
Time: 2022-02-25 08:38:28
-------------------------------------------
stream counter=34902.0
stream counter=34903.0
stream counter=34904.0
stream counter=34905.0
stream counter=34906.0
stream counter=34907.0
stream counter=34908.0
stream counter=34909.0
stream counter=34910.0
stream counter=34911.0
...

-------------------------------------------
Time: 2022-02-25 08:38:28
-------------------------------------------
rls1: beta=[[ 4.98494995e-02]
 [ 1.00026736e+00]
 [-7.47196564e-04]
 [ 3.69720325e-04]
 [ 2.24089809e-04]
 [ 9.71682314e-05]
 [ 1.80307661e-04]
 [-7.43128111e-06]
 [-1.13364040e-04]
 [ 5.33198880e-04]
 [ 1.00001219e+00]]
rls2: beta=[[ 5.44725002e-02]
 [ 9.97033691e-01]
 [-3.03084676e-03]
 [-1.89396566e-03]
 [ 1.73192061e-03]
 [ 1.62853729e-03]
 [-6.74402916e-04]
 [-4.95836266e-04]
 [ 7.61814034e-04]
 [ 5.60195785e-03]
 [ 9.96399027e-01]]

-------------------------------------------
Time: 2022-02-25 08:38:28
------------------------

-------------------------------------------
Time: 2022-02-25 08:38:30
-------------------------------------------
stream counter=35074.0
stream counter=35075.0
stream counter=35076.0
stream counter=35077.0
stream counter=35078.0
stream counter=35079.0
stream counter=35080.0
stream counter=35081.0
stream counter=35082.0
stream counter=35083.0
...

-------------------------------------------
Time: 2022-02-25 08:38:30
-------------------------------------------
rls1: beta=[[ 4.99038127e-02]
 [ 1.00024904e+00]
 [-7.44087462e-04]
 [ 3.62563231e-04]
 [ 1.99128307e-04]
 [ 7.88934628e-05]
 [ 2.18946785e-04]
 [-1.92742080e-05]
 [-2.99699799e-05]
 [ 4.95229977e-04]
 [ 1.00004715e+00]]
rls2: beta=[[ 5.15963004e-02]
 [ 1.00038889e+00]
 [-3.13575262e-04]
 [ 4.48949266e-04]
 [-4.35563585e-04]
 [ 3.39893155e-04]
 [ 1.31730049e-03]
 [ 2.02534099e-04]
 [ 6.51655659e-03]
 [-1.66013915e-03]
 [ 1.00160749e+00]]

-------------------------------------------
Time: 2022-02-25 08:38:30
------------------------

-------------------------------------------
Time: 2022-02-25 08:38:32
-------------------------------------------
stream counter=35250.0
stream counter=35251.0
stream counter=35252.0
stream counter=35253.0
stream counter=35254.0
stream counter=35255.0
stream counter=35256.0
stream counter=35257.0
stream counter=35258.0
stream counter=35259.0
...

-------------------------------------------
Time: 2022-02-25 08:38:32
-------------------------------------------
rls1: beta=[[ 4.98858445e-02]
 [ 1.00020251e+00]
 [-6.78875733e-04]
 [ 2.78518773e-04]
 [ 1.98219167e-04]
 [ 2.90713598e-06]
 [ 1.48800873e-04]
 [-6.95019459e-05]
 [-7.34101489e-05]
 [ 5.48676179e-04]
 [ 1.00005388e+00]]
rls2: beta=[[ 4.91313992e-02]
 [ 9.98912201e-01]
 [ 2.33398821e-03]
 [-3.49722592e-03]
 [-2.08112060e-04]
 [-3.49300407e-03]
 [-1.86030363e-03]
 [ 1.86730226e-05]
 [-2.06881963e-04]
 [ 2.44503359e-03]
 [ 1.00102503e+00]]

-------------------------------------------
Time: 2022-02-25 08:38:32
------------------------

In [16]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 08:38:34 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


-------------------------------------------
Time: 2022-02-25 08:38:34
-------------------------------------------
stream counter=35424.0
stream counter=35425.0
stream counter=35426.0
stream counter=35427.0
stream counter=35428.0
stream counter=35429.0
stream counter=35430.0
stream counter=35431.0
stream counter=35432.0
stream counter=35433.0
...



Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurren

-------------------------------------------
Time: 2022-02-25 08:38:34
-------------------------------------------
rls1: beta=[[ 4.98633331e-02]
 [ 1.00020661e+00]
 [-6.89653946e-04]
 [ 2.65616884e-04]
 [ 2.49730635e-04]
 [-3.52031847e-05]
 [ 1.91986526e-04]
 [-1.22841160e-04]
 [-1.49881437e-04]
 [ 4.67547790e-04]
 [ 1.00010784e+00]]
rls2: beta=[[ 4.75574573e-02]
 [ 1.00006620e+00]
 [-8.69078385e-04]
 [-3.22356850e-03]
 [ 3.41987734e-03]
 [-2.82506410e-03]
 [ 1.78826528e-03]
 [-4.04941972e-03]
 [-3.55486986e-03]
 [-3.63621762e-03]
 [ 1.00267214e+00]]

-------------------------------------------
Time: 2022-02-25 08:38:34
-------------------------------------------
rls1: mse=[[1.45136619e-05]]
rls2: mse=[[1.49971298e-05]]

-------------------------------------------
Time: 2022-02-25 08:38:34
-------------------------------------------
rls1: N=8910.0
rls2: N=8910.0

-------------------------------------------
Time: 2022-02-25 08:38:34
-------------------------------------------
rls1: y=-2.

# ML batch predictor (racing of several models) 

To conclude this session, below you can find an example of the implementation of the parallel racing of multiple machine learning predictors (here *Random Forests* - `rf`, *Linear Model* - `lin`, *Gradient boosting* - `gb`, respectively).

As you can observe, the general structure of the state and the `updateFunction` is the same as the previous section, with:

* The *state* is modified in order to include the states of the three models.
    A state is a list of 4 elements:
    * `D` - The data batch to predict
    * `mse` - Current value for the MSE
    * `N` - Counter of the values 
    * `E` - Vector of historical error measues
   The state of each model is then combined in a key-value RDD with a key identifying the type of model (cf. `state1,state2,state3`).
   
* The original Dstream is duplicated using a `flatMap` operation in order to create one key-pair streams for each considered model 

* The *update function* is modified to employ the predictors from Scikit-learn to predict the last value for the batch (cf. `predict` function definition), in a leave-one-out fashion, with the training of the predictor being performed on the batch of previously collected data (`DD` variable).

* Note that there is only a single `updateFunction`, since the `updateStateByKey` operation applies the update function in parallel to all the different keys (in this case, each key correspond to a model).


In [44]:
n=10 # number of features

batchSize=50
D=np.zeros((batchSize,n+1)) 
## batch data size
E=np.zeros((1,1))
mse=0
N=0
state1=('rf',D,mse,N,E,0,0)
state2=('lin',D,mse,N,E,0,0)
state3=('gb',D,mse,N,E,0,0)


initialStateRDD = sc.parallelize([(u'rf', state1),
                                  (u'lin', state2),
                                 (u'gb', state3)])

In [45]:
def predict(mod,YX,q):
    N=YX.shape[0]
    Ntr=int(np.round(N/2))
    Ytr=YX[:Ntr,0]
    Xtr=YX[:Ntr,1:]
    Yts=YX[Ntr:,0]
    Xts=YX[Ntr:,1:]
    Nts=Xts.shape[0]
    if mod=="rf":
        regr = RandomForestRegressor(random_state=0)
    if mod=="gb":    
        regr = GradientBoostingRegressor(random_state=0)
    if mod=="lin":
        regr=LinearRegression()
    regr.fit(Xtr, Ytr)
    Yhat=regr.predict(q)
   

    return(Yhat)

In [46]:
def updateFunction(new_values, state): 
    L=len(new_values)
    
    if (L>0):
        # Extract the data from state variable
        (model_key,DD,mse,N,E,y,yhat) = state
        #model_key=state[0] ## Unique identifier for the model
        #DD=state[1]  ## Analyzed values
        #mse=state[2]  ## Mean squared errors
        #N=state[3]   ## Number of treated samples
        #E=state[4]   ## Error measure
        #y=state[5]   ## True value for the last value in the batch
        #yhat=state[6] ## Predicted value for the last value in the batch

        key=new_values[0][0]
          
        ## prediction of the output of the most recent value
        yx=new_values[-1][1]      
        i=yx[0]
        y=yx[1]
        x=yx[2:]
        x.shape=(1,n)
        N=N+1
        
        yhat=predict(key,DD,x) 
        err=y-yhat  ## prediction error for the latest values
        mse=mse+1.0/N*(pow(err,2.0)-mse) ## sequential update of MSE
        E=np.append(E,err)        
        
        ## batch update
        for l in np.arange(L):
            yx=new_values[l][1]      
            d=yx[1:]
            d.shape=(1,n+1)
            DD=np.vstack((d,DD[:-1,:]))
        return (key,DD,mse,N,E,y,yhat)  
        
    else:
        return state

In [47]:
[sc,ssc,dstream]=getDStream(spark=spark,batch_interval=2)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x)))

dataS.map(lambda x: "stream counter="+ str(x[0])).pprint()

dataS2=dataS.flatMap(lambda x: [('rf',('rf',1.0*np.array(x))),
                            ('lin',('lin',1.0*np.array(x))),
                            ('gb',('gb',1.0*np.array(x)))])
updatedS=dataS2.updateStateByKey(updateFunction,initialRDD=initialStateRDD)

## printing out updated values of the states of the two models
outmseS=updatedS.map(lambda x: x[1][0]+": mse="+ str(x[1][2])).pprint()
#outNS=updatedS.map(lambda x: x[1][0]+": N="+ str(x[1][3])+ " D shape="+ str(x[1][1].shape)).pprint()
#outyS=updatedS.map(lambda x: x[1][0]+": y="+ str(x[1][5])+ " yhat="+ str(x[1][6])).pprint()


In [48]:
ssc.start()

-------------------------------------------
Time: 2022-02-25 08:58:58
-------------------------------------------
stream counter=60313.0
stream counter=60314.0
stream counter=60315.0
stream counter=60316.0
stream counter=60317.0
stream counter=60318.0
stream counter=60319.0
stream counter=60320.0
stream counter=60321.0
stream counter=60322.0
...

-------------------------------------------
Time: 2022-02-25 08:58:58
-------------------------------------------
gb: mse=[0.20943204]
lin: mse=[0.20943204]
rf: mse=[0.20943204]



-------------------------------------------
Time: 2022-02-25 08:59:00
-------------------------------------------
stream counter=60381.0
stream counter=60382.0
stream counter=60383.0
stream counter=60384.0
stream counter=60385.0
stream counter=60386.0
stream counter=60387.0
stream counter=60388.0
stream counter=60389.0
stream counter=60390.0
...

-------------------------------------------
Time: 2022-02-25 08:59:00
-------------------------------------------
gb: mse=[0.11512516]
lin: mse=[0.10506088]
rf: mse=[0.13677168]



-------------------------------------------
Time: 2022-02-25 08:59:02
-------------------------------------------
stream counter=60553.0
stream counter=60554.0
stream counter=60555.0
stream counter=60556.0
stream counter=60557.0
stream counter=60558.0
stream counter=60559.0
stream counter=60560.0
stream counter=60561.0
stream counter=60562.0
...

-------------------------------------------
Time: 2022-02-25 08:59:02
-------------------------------------------
gb: mse=[0.08009613]
lin: mse=[0.07032985]
rf: mse=[0.0956807]



-------------------------------------------
Time: 2022-02-25 08:59:04
-------------------------------------------
stream counter=60727.0
stream counter=60728.0
stream counter=60729.0
stream counter=60730.0
stream counter=60731.0
stream counter=60732.0
stream counter=60733.0
stream counter=60734.0
stream counter=60735.0
stream counter=60736.0
...

-------------------------------------------
Time: 2022-02-25 08:59:04
-------------------------------------------
gb: mse=[0.22730077]
lin: mse=[0.05285777]
rf: mse=[0.17571122]



-------------------------------------------
Time: 2022-02-25 08:59:06
-------------------------------------------
stream counter=60901.0
stream counter=60902.0
stream counter=60903.0
stream counter=60904.0
stream counter=60905.0
stream counter=60906.0
stream counter=60907.0
stream counter=60908.0
stream counter=60909.0
stream counter=60910.0
...

-------------------------------------------
Time: 2022-02-25 08:59:06
-------------------------------------------
gb: mse=[0.23819107]
lin: mse=[0.04236263]
rf: mse=[0.30248459]



...

-------------------------------------------
Time: 2022-02-25 08:59:54
-------------------------------------------
stream counter=65078.0
stream counter=65079.0
stream counter=65080.0
stream counter=65081.0
stream counter=65082.0
stream counter=65083.0
stream counter=65084.0
stream counter=65085.0
stream counter=65086.0
stream counter=65087.0
...

-------------------------------------------
Time: 2022-02-25 08:59:54
-------------------------------------------
gb: mse=[0.55618733]
lin: mse=[0.00845173]
rf: mse=[0.75747789]



-------------------------------------------
Time: 2022-02-25 08:59:56
-------------------------------------------
stream counter=65253.0
stream counter=65254.0
stream counter=65255.0
stream counter=65256.0
stream counter=65257.0
stream counter=65258.0
stream counter=65259.0
stream counter=65260.0
stream counter=65261.0
stream counter=65262.0
...

-------------------------------------------
Time: 2022-02-25 08:59:56
-------------------------------------------
gb: mse=[0.67477784]
lin: mse=[0.0082746]
rf: mse=[0.86109044]



-------------------------------------------
Time: 2022-02-25 08:59:58
-------------------------------------------
stream counter=65424.0
stream counter=65425.0
stream counter=65426.0
stream counter=65427.0
stream counter=65428.0
stream counter=65429.0
stream counter=65430.0
stream counter=65431.0
stream counter=65432.0
stream counter=65433.0
...

-------------------------------------------
Time: 2022-02-25 08:59:58
-------------------------------------------
gb: mse=[0.65529608]
lin: mse=[0.00800771]
rf: mse=[0.84210103]



-------------------------------------------
Time: 2022-02-25 09:00:00
-------------------------------------------
stream counter=65601.0
stream counter=65602.0
stream counter=65603.0
stream counter=65604.0
stream counter=65605.0
stream counter=65606.0
stream counter=65607.0
stream counter=65608.0
stream counter=65609.0
stream counter=65610.0
...

-------------------------------------------
Time: 2022-02-25 09:00:00
-------------------------------------------
gb: mse=[0.634842]
lin: mse=[0.00776559]
rf: mse=[0.82829042]



-------------------------------------------
Time: 2022-02-25 09:00:02
-------------------------------------------
stream counter=65772.0
stream counter=65773.0
stream counter=65774.0
stream counter=65775.0
stream counter=65776.0
stream counter=65777.0
stream counter=65778.0
stream counter=65779.0
stream counter=65780.0
stream counter=65781.0
...

-------------------------------------------
Time: 2022-02-25 09:00:02
-------------------------------------------
gb: mse=[0.61933913]
lin: mse=[0.00760275]
rf: mse=[0.82983093]



Note that the batch is empty at the beginning. So the prediction becomes more accurate (and the MSE smaller) only when  the batch dataset if filled.

In [49]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

22/02/25 09:00:03 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
Exception in thread "receiver-supervisor-future-0" java.lang.Error: java.lang.InterruptedException: sleep interrupted
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1155)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.InterruptedException: sleep interrupted
	at java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:3

# Last but not least: debugging 

The main difficulty in programming streaming analytics code is the debugging of the code. Let's see an example

In [ ]:
def WRONGupdateFunction(new_values, state): 
    L=len(new_values)
    ## size buffer
    if (L>0):
        ## value=new_values[0] ## CORRECT: take the first value
        value=new_values   ## WRONG
        d=np.abs(value[1:]).reshape(1,-1)+state[0] 
        return (d,L)  
        
    else:
        return state

In [ ]:
[sc,ssc,dstream]=getKafkaDStream(spark=spark,topic='dataLinearModel',batch_interval=2)
dataS = dstream.map(lambda x: np.array(ast.literal_eval(x[1])))
dataS.map(lambda x: "stream counter="+ str(x[0])).pprint()

dataS=dataS.flatMap(lambda x: [('state',1.0*np.array(x))])

n=11
v=np.zeros((1,n))

state1=(v,0)
## state with two components: vector of size 11 and an integer

initialStateRDD = sc.parallelize([('state', state1)])

updatedS=dataS.updateStateByKey(WRONGupdateFunction,initialRDD=initialStateRDD)
updatedS.pprint()

In [ ]:
ssc.start()

If you run the wrong code you will see on the **notebook outbook** only the output of `pprint()` command but not the second output. 

At the same time if you go to the **terminal window** you will see a long series of messages like:

```
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:456)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:592)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:575)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:410)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at org.apache.spark.storage.memory.MemoryStore.putIterator(MemoryStore.scala:221)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:299)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.apply(BlockManager.scala:1165)
	at org.apache.spark.storage.BlockManager$$anonfun$doPutIterator$1.apply(BlockManager.scala:1156)
	at org.apache.spark.storage.BlockManager.doPut(BlockManager.scala:1091)
	at org.apache.spark.storage.BlockManager.doPutIterator(BlockManager.scala:1156)
	at org.apache.spark.storage.BlockManager.getOrElseUpdate(BlockManager.scala:882)
	at org.apache.spark.rdd.RDD.getOrCompute(RDD.scala:357)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:308)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:346)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:310)
	at org.apache.spark.rdd.PartitionerAwareUnionRDD$$anonfun$compute$1.apply(PartitionerAwareUnionRDD.scala:100)
	at org.apache.spark.rdd.PartitionerAwareUnionRDD$$anonfun$compute$1.apply(PartitionerAwareUnionRDD.scala:99)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:435)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:441)
```

Most of those messages are very _low level_ and of no help but some refer to your actual code like
   
  **File "<ipython-input-141-72bbbd66e1c1>", line 7, in WRONGupdateFunction**
    
**ValueError: operands could not be broadcast together with shapes (1,372) (1,11)**

```
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:456)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:592)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:575)
```

The problem was indeed in line 7 of your update function...
Correct it and launch again..

In [ ]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)